In [1]:
from minio import Minio
import findspark
import os
import psycopg2

findspark.init()
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import (
    regexp_extract,
    col,
    when,
    length,
    year,
    month,
    quarter,
    dayofweek,
    weekofyear,
    dayofmonth,
    datediff,
    to_date,
    to_timestamp,
    unix_timestamp,
    lit,
    lower,
    current_date,
    coalesce,
    sum as spark_sum,
    avg as spark_avg,
    max as spark_max,
    min as spark_min,
    floor,
    count,
    greatest,
    expr,
    row_number,
    countDistinct,
    lag
)
from pyspark.sql.types import (
    NumericType,
    IntegerType,
    LongType,
    FloatType,
    DoubleType,
    DecimalType,
)
from pyspark.sql.window import Window
from pyspark.ml.feature import Imputer
import pyspark.sql.functions as F
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
spark = (
    SparkSession.builder.appName("Transformation")
    .master("local[*]")
    .config("spark.dynamicAllocation.enabled", "true")
    .config("spark.dynamicAllocation.minExecutors", "0")
    .config("spark.dynamicAllocation.maxExecutors", "8")
    .config("spark.dynamicAllocation.initialExecutors", "1")
    # S3A/MinIO JAR dependencies for PySpark 3.5.0
    .config(
        "spark.jars.packages",
        "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262,org.postgresql:postgresql:42.5.0",
    )
    # S3A/MinIO configuration
    .config("spark.hadoop.fs.s3a.endpoint", os.getenv("MINIO_ENDPOINT"))
    .config("spark.hadoop.fs.s3a.access.key", os.getenv("MINIO_ACCESS_KEY"))
    .config("spark.hadoop.fs.s3a.secret.key", os.getenv("MINIO_SECRET_KEY"))
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config(
        "spark.hadoop.fs.s3a.aws.credentials.provider",
        "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider",
    )
    .config("inferSchema", "true")
    .config("mergeSchema", "true")
    .getOrCreate()
)

25/11/27 23:43:30 WARN Utils: Your hostname, KD4SH1-BFMG resolves to a loopback address: 127.0.1.1; using 192.168.100.195 instead (on interface wlan0)
25/11/27 23:43:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/khalid_ah_1/.local/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/khalid_ah_1/.ivy2/cache
The jars for the packages stored in: /home/khalid_ah_1/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b143b129-3fcf-48b9-b8b7-943d3d7200f2;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found org.postgresql#postgresql;42.5.0 in central
	found org.checkerframework#checker-qual;3.5.0 in central
:: resolution report :: resolve 338ms :: artifacts dl 10ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.checkerframework#checker-qual;3.5.0 from central in [default]
	org.postgresql#postgresql;42.5

In [3]:
minio_client = Minio(
    os.getenv("MINIO_ENDPOINT"),
    access_key=os.getenv("MINIO_ACCESS_KEY"),
    secret_key=os.getenv("MINIO_SECRET_KEY"),
    secure=False,
)

bucket_name = "pulse-bucket-1"

In [4]:
objects = minio_client.list_objects(bucket_name, prefix="cleaned_", recursive=True)
dataframes = {}
for obj in objects:
    df = spark.read.csv(
        f"s3a://{bucket_name}/{obj.object_name}", header=True, inferSchema=True
    )
    object_name = obj.object_name.replace("cleaned_", "").replace(".csv", "")
    dataframes[object_name] = df

25/11/27 23:43:35 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [5]:
dataframes

{'customer_sessions': DataFrame[session_id: string, customer_id: int, session_start: timestamp, session_end: timestamp, device_type: string, referrer_source: string, pages_viewed: int, products_viewed: int, conversion_flag: boolean, cart_abandonment_flag: boolean],
 'customers': DataFrame[customer_id: int, gender: string, date_of_birth: date, account_status: string, city: string, state_province: string, postal_code: string, country: string, account_created_at: timestamp, last_login_date: date, is_active: boolean],
 'inventory': DataFrame[inventory_id: int, product_id: int, supplier_id: int, stock_quantity: int, reserved_quantity: int, minimum_stock_level: int, last_restocked_date: date, storage_cost: double],
 'marketing_campaigns': DataFrame[campaign_id: int, campaign_name: string, campaign_type: string, start_date: date, end_date: date, budget: double, spent_amount: double, impressions: int, clicks: int, conversions: int, target_audience: string, campaign_status: string],
 'order_ite

# Transformations

In [6]:
total_sold_df = dataframes["order_items"].groupBy("product_id").agg(
    spark_sum("quantity").alias("total_sold")
)

In [7]:
avg_inventory_df = dataframes["inventory"].groupBy("product_id").agg(
    spark_avg("stock_quantity").alias("avg_inventory")
)

In [8]:
avg_daily_sales = (
    dataframes["orders"]
    .groupBy(to_date("order_placed_at").alias("order_date"))
    .agg(spark_sum("total_amount").alias("daily_sales"))
    .select(spark_avg("daily_sales").alias("avg_daily_sales"))
    .collect()[0]["avg_daily_sales"]
)

25/11/27 23:43:44 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [9]:
placed_timestamp = to_timestamp(col("order_placed_at"))
shipped_timestamp = to_timestamp(col("order_shipped_at"))
delivered_timestamp = to_timestamp(col("order_delivered_at"))

order_metrics = (
    dataframes["order_items"]
    .groupBy("order_id")
    .agg(
        spark_sum("product_cost").alias("total_product_cost"),
        spark_sum("quantity").alias("total_quantity"),
        spark_avg("product_cost").alias("avg_product_cost"),
        spark_max("discount_amount").alias("max_item_discount"),
        countDistinct("product_id").alias("unique_products_ordered"),
    )
)

dataframes["orders"] = (
    dataframes["orders"]
    .join(
        order_metrics,
        "order_id",
        "inner",
    )
    .withColumns(
        {
            "order_placed_year": when(
                col("order_placed_at").isNotNull(), year(col("order_placed_at"))
            ),
            "order_placed_month": when(
                col("order_placed_at").isNotNull(), month(col("order_placed_at"))
            ),
            "order_placed_quarter": when(
                col("order_placed_at").isNotNull(), quarter(col("order_placed_at"))
            ),
            "order_placed_day_of_week": when(
                col("order_placed_at").isNotNull(), dayofweek(col("order_placed_at"))
            ),
            "order_placed_week_of_year": when(
                col("order_placed_at").isNotNull(), weekofyear(col("order_placed_at"))
            ),
            "order_placed_day_of_month": when(
                col("order_placed_at").isNotNull(), dayofmonth(col("order_placed_at"))
            ),
            "order_shipped_year": when(
                col("order_shipped_at").isNotNull(), year(col("order_shipped_at"))
            ),
            "order_shipped_month": when(
                col("order_shipped_at").isNotNull(), month(col("order_shipped_at"))
            ),
            "order_shipped_quarter": when(
                col("order_shipped_at").isNotNull(), quarter(col("order_shipped_at"))
            ),
            "order_shipped_day_of_week": when(
                col("order_shipped_at").isNotNull(), dayofweek(col("order_shipped_at"))
            ),
            "order_shipped_week_of_year": when(
                col("order_shipped_at").isNotNull(), weekofyear(col("order_shipped_at"))
            ),
            "order_shipped_day_of_month": when(
                col("order_shipped_at").isNotNull(), dayofmonth(col("order_shipped_at"))
            ),
            "order_delivered_year": when(
                col("order_delivered_at").isNotNull(), year(col("order_delivered_at"))
            ),
            "order_delivered_month": when(
                col("order_delivered_at").isNotNull(), month(col("order_delivered_at"))
            ),
            "order_delivered_quarter": when(
                col("order_delivered_at").isNotNull(),
                quarter(col("order_delivered_at")),
            ),
            "order_delivered_day_of_week": when(
                col("order_delivered_at").isNotNull(),
                dayofweek(col("order_delivered_at")),
            ),
            "order_delivered_week_of_year": when(
                col("order_delivered_at").isNotNull(),
                weekofyear(col("order_delivered_at")),
            ),
            "order_delivered_day_of_month": when(
                col("order_delivered_at").isNotNull(),
                dayofmonth(col("order_delivered_at")),
            ),
            "order_processing_seconds_diff": when(
                placed_timestamp.isNotNull() & shipped_timestamp.isNotNull(),
                greatest(
                    unix_timestamp(shipped_timestamp)
                    - unix_timestamp(placed_timestamp),
                    lit(0),
                ),
            ),
            "order_processing_minutes_diff": when(
                placed_timestamp.isNotNull() & shipped_timestamp.isNotNull(),
                greatest(
                    floor(
                        (
                            unix_timestamp(shipped_timestamp)
                            - unix_timestamp(placed_timestamp)
                        )
                        / 60
                    ),
                    lit(0),
                ),
            ),
            "order_processing_hours_diff": when(
                placed_timestamp.isNotNull() & shipped_timestamp.isNotNull(),
                greatest(
                    floor(
                        (
                            unix_timestamp(shipped_timestamp)
                            - unix_timestamp(placed_timestamp)
                        )
                        / 3600
                    ),
                    lit(0),
                ),
            ),
            "order_processing_days_diff": when(
                placed_timestamp.isNotNull() & shipped_timestamp.isNotNull(),
                greatest(datediff(shipped_timestamp, placed_timestamp), lit(0)),
            ),
            "order_processing_weeks_diff": when(
                placed_timestamp.isNotNull() & shipped_timestamp.isNotNull(),
                greatest(datediff(shipped_timestamp, placed_timestamp) / 7, lit(0)),
            ),
            "order_processing_months_diff": when(
                placed_timestamp.isNotNull() & shipped_timestamp.isNotNull(),
                greatest(datediff(shipped_timestamp, placed_timestamp) / 30, lit(0)),
            ),
            "order_processing_years_diff": when(
                placed_timestamp.isNotNull() & shipped_timestamp.isNotNull(),
                greatest(datediff(shipped_timestamp, placed_timestamp) / 365, lit(0)),
            ),
            "delivery_seconds_diff": when(
                shipped_timestamp.isNotNull() & delivered_timestamp.isNotNull(),
                greatest(
                    unix_timestamp(delivered_timestamp)
                    - unix_timestamp(shipped_timestamp),
                    lit(0),
                ),
            ),
            "delivery_minutes_diff": when(
                shipped_timestamp.isNotNull() & delivered_timestamp.isNotNull(),
                greatest(
                    floor(
                        (
                            unix_timestamp(delivered_timestamp)
                            - unix_timestamp(shipped_timestamp)
                        )
                        / 60
                    ),
                    lit(0),
                ),
            ),
            "delivery_hours_diff": when(
                shipped_timestamp.isNotNull() & delivered_timestamp.isNotNull(),
                greatest(
                    floor(
                        (
                            unix_timestamp(delivered_timestamp)
                            - unix_timestamp(shipped_timestamp)
                        )
                        / 3600
                    ),
                    lit(0),
                ),
            ),
            "delivery_days_diff": when(
                shipped_timestamp.isNotNull() & delivered_timestamp.isNotNull(),
                greatest(datediff(delivered_timestamp, shipped_timestamp), lit(0)),
            ),
            "delivery_weeks_diff": when(
                shipped_timestamp.isNotNull() & delivered_timestamp.isNotNull(),
                greatest(datediff(delivered_timestamp, shipped_timestamp) / 7, lit(0)),
            ),
            "delivery_months_diff": when(
                shipped_timestamp.isNotNull() & delivered_timestamp.isNotNull(),
                greatest(datediff(delivered_timestamp, shipped_timestamp) / 30, lit(0)),
            ),
            "delivery_years_diff": when(
                shipped_timestamp.isNotNull() & delivered_timestamp.isNotNull(),
                greatest(
                    datediff(delivered_timestamp, shipped_timestamp) / 365, lit(0)
                ),
            ),
            "total_order_fulfillment_time_seconds": when(
                col("order_processing_seconds_diff").isNotNull()
                & col("delivery_seconds_diff").isNotNull(),
                col("order_processing_seconds_diff") + col("delivery_seconds_diff"),
            ),
            "total_order_fulfillment_time_minutes": when(
                col("order_processing_minutes_diff").isNotNull()
                & col("delivery_minutes_diff").isNotNull(),
                col("order_processing_minutes_diff") + col("delivery_minutes_diff"),
            ),
            "total_order_fulfillment_time_hours": when(
                col("order_processing_hours_diff").isNotNull()
                & col("delivery_hours_diff").isNotNull(),
                col("order_processing_hours_diff") + col("delivery_hours_diff"),
            ),
            "total_order_fulfillment_time_days": when(
                col("order_processing_days_diff").isNotNull()
                & col("delivery_days_diff").isNotNull(),
                col("order_processing_days_diff") + col("delivery_days_diff"),
            ),
            "total_order_fulfillment_time_weeks": when(
                col("order_processing_weeks_diff").isNotNull()
                & col("delivery_weeks_diff").isNotNull(),
                col("order_processing_weeks_diff") + col("delivery_weeks_diff"),
            ),
            "total_order_fulfillment_time_months": when(
                col("order_processing_months_diff").isNotNull()
                & col("delivery_months_diff").isNotNull(),
                col("order_processing_months_diff") + col("delivery_months_diff"),
            ),
            "total_order_fulfillment_time_years": when(
                col("order_processing_years_diff").isNotNull()
                & col("delivery_years_diff").isNotNull(),
                col("order_processing_years_diff") + col("delivery_years_diff"),
            ),
            "order_profit": when(
                col("subtotal").isNotNull()
                & col("total_product_cost").isNotNull()
                & col("total_quantity").isNotNull()
                & (col("subtotal") != 0)
                & (col("total_quantity") != 0)
                & (col("total_product_cost") != 0),
                col("subtotal") - (col("total_product_cost") * col("total_quantity")),
            ),
            "net_revenue": when(
                col("total_amount").isNotNull()
                & col("total_discount").isNotNull()
                & col("shipping_cost").isNotNull()
                & (col("total_amount") != 0)
                & (col("shipping_cost") != 0)
                & (col("total_discount") != 0),
                col("total_amount") - col("total_discount") - col("shipping_cost"),
            ),
            "net_profit": when(
                col("order_profit").isNotNull()
                & col("shipping_cost").isNotNull()
                & (col("order_profit") != 0)
                & (col("shipping_cost") != 0),
                col("order_profit") - col("shipping_cost"),
            ),
            "discount_percentage": when(
                col("subtotal").isNotNull()
                & col("total_discount").isNotNull()
                & (col("subtotal") != 0)
                & (col("total_discount") != 0),
                (col("total_discount") / col("subtotal")) * 100,
            ),
            "average_item_value": when(
                col("subtotal").isNotNull()
                & col("total_quantity").isNotNull()
                & (col("subtotal") != 0)
                & (col("total_quantity") != 0),
                col("subtotal") / col("total_quantity"),
            ),
            "cost_per_item": when(
                col("total_product_cost").isNotNull()
                & col("total_quantity").isNotNull()
                & (col("total_product_cost") != 0)
                & (col("total_quantity") != 0),
                col("total_product_cost") / col("total_quantity"),
            ),
            "order_size_category": when(
                col("total_quantity").isNotNull() & (col("total_quantity") != 0),
                when(col("total_quantity") < 3, "Small")
                .when(
                    (col("total_quantity") >= 3) & (col("total_quantity") < 7), "Medium"
                )
                .otherwise("Large"),
            ),
            "season": when(
                col("order_placed_month").isNotNull(),
                when(col("order_placed_month").isin(12, 1, 2), "Winter")
                .when(col("order_placed_month").isin(3, 4, 5), "Spring")
                .when(col("order_placed_month").isin(6, 7, 8), "Summer")
                .otherwise("Fall"),
            ),
        }
    )
).dropDuplicates(["order_id"])

In [10]:
dataframes["orders"].show(5)

25/11/27 23:43:46 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+-----------+------------+--------+----------+-------------+--------------+------------+--------+-------------------+----------------+------------------+------------------+--------------+------------------+-----------------+-----------------------+-----------------+------------------+--------------------+------------------------+-------------------------+-------------------------+------------------+-------------------+---------------------+-------------------------+--------------------------+--------------------------+--------------------+---------------------+-----------------------+---------------------------+----------------------------+----------------------------+-----------------------------+-----------------------------+---------------------------+--------------------------+---------------------------+----------------------------+---------------------------+---------------------+---------------------+-------------------+------------------+-------------------+-----------

In [11]:
customers_with_orders = dataframes["customers"] \
    .drop("order_recency_days", "order_frequency", "order_total_spent") \
    .join(dataframes["orders"].select("customer_id", "order_placed_at", "order_id", "total_amount").groupBy("customer_id").agg(
        datediff(current_date(), spark_max("order_placed_at")).alias("order_recency_days"),
        count("order_id").alias("order_frequency"),
        spark_sum("total_amount").alias("order_total_spent"),
    ), "customer_id", "left")

dataframes["customers"] = customers_with_orders.withColumns(
    {
        "customer_age": when(col("date_of_birth").isNotNull(), floor(datediff(current_date(), col("date_of_birth")) / 365)),
        "customer_tenure_days": when(col("account_created_at").isNotNull(), datediff(current_date(), to_date(col("account_created_at")))),
        "days_since_last_login": when(col("last_login_date").isNotNull(), datediff(current_date(), to_date(col("last_login_date")))),
        "customer_age_group": when(
            col("customer_age").isNotNull(),
            when(col("customer_age") < 18, "Under 18")
            .when((col("customer_age") >= 18) & (col("customer_age") < 25), "18-24")
            .when((col("customer_age") >= 25) & (col("customer_age") < 35), "25-34")
            .when((col("customer_age") >= 35) & (col("customer_age") < 45), "35-44")
            .when((col("customer_age") >= 45) & (col("customer_age") < 55), "45-54")
            .when((col("customer_age") >= 55) & (col("customer_age") < 65), "55-64")
            .otherwise("65 and over")
        ),
        "customer_activity_status": when(
            col("days_since_last_login").isNotNull(),
            when(col("days_since_last_login") <= 30, "Active")
            .when((col("days_since_last_login") > 30) & (col("days_since_last_login") <= 90), "Inactive")
            .otherwise("Dormant")
        ),
        "customer_segment": when(
            col("order_recency_days").isNotNull(),
            when((col("order_frequency") >= 10) & (col("order_total_spent") >= 1000) & (col("order_recency_days") <= 30), "High Value")
            .when((col("order_frequency") >= 5) & (col("order_total_spent") >= 500) & (col("order_recency_days") <= 60), "Medium Value")
            .when((col("order_frequency") >= 2) & (col("order_total_spent") >= 100) & (col("order_recency_days") <= 90), "Low Value")
            .when((col("order_frequency") == 1) & (col("order_recency_days") <= 30), "New")
            .when((col("order_recency_days") > 90) & (col("order_recency_days") <= 180), "At Risk")
            .when(col("order_recency_days") > 180, "Lost")
            .otherwise("Uncategorized")
        ),
        "customer_lifetime_value": (
            (col("order_total_spent") / when(col("order_frequency") > 0, col("order_frequency")).otherwise(lit(1)))
            *
            ((col("order_frequency") / when(col("customer_tenure_days") > 0, col("customer_tenure_days")).otherwise(lit(365))) * 365)
            *
            lit(3.0) # Expected lifespan in years
        )
    }
)

In [12]:
dataframes["customers"].show(5)

+-----------+------+-------------+--------------+--------------+------------------+-----------+----------+-------------------+---------------+---------+------------------+---------------+------------------+------------+--------------------+---------------------+------------------+------------------------+----------------+-----------------------+
|customer_id|gender|date_of_birth|account_status|          city|    state_province|postal_code|   country| account_created_at|last_login_date|is_active|order_recency_days|order_frequency| order_total_spent|customer_age|customer_tenure_days|days_since_last_login|customer_age_group|customer_activity_status|customer_segment|customer_lifetime_value|
+-----------+------+-------------+--------------+--------------+------------------+-----------+----------+-------------------+---------------+---------+------------------+---------------+------------------+------------+--------------------+---------------------+------------------+-----------------------

In [13]:
dataframes["marketing_campaigns"].show(5)

+-----------+--------------------+-------------+----------+----------+--------+------------+-----------+------+-----------+--------------------+---------------+
|campaign_id|       campaign_name|campaign_type|start_date|  end_date|  budget|spent_amount|impressions|clicks|conversions|     target_audience|campaign_status|
+-----------+--------------------+-------------+----------+----------+--------+------------+-----------+------+-----------+--------------------+---------------+
|        158|Referral Program ...|          PPC|2025-11-16|2025-11-20| 17868.2|       881.8|      78302|  3319|         60|     Budget shoppers|         Active|
|        229|Flash Sale 2025 -...|  Direct Mail|2025-11-16|2025-11-20|18523.86|     1188.83|      72217|  2885|        112|Parents with chil...|           NULL|
|        297|Spring Collection...|          SMS|2025-11-16|2025-11-20| 7829.94|       199.6|       6027|    48|          4|Millennials, Tech...|           NULL|
|        384|VIP Exclusive 202...|

In [14]:
dataframes["marketing_campaigns"] = dataframes["marketing_campaigns"] \
    .join(
        dataframes["customer_sessions"]
        .join(
            dataframes["marketing_campaigns"],
            (col("session_start") >= col("start_date")) & 
            (col("session_start") <= col("end_date")),
            "cross"
        )
        .select(
            col("session_id"),
            col("customer_id"),
            col("conversion_flag"),
            col("campaign_id"),
            col("start_date").alias("campaign_start"),
            col("end_date").alias("campaign_end")
        )
        .filter(col("conversion_flag") == "true")
        .join(
            dataframes["orders"].filter(
                (col("order_status") != "cancelled") & 
                (col("order_status") != "refunded")
            ),
            "customer_id",
            "inner"
        )
        # Add time constraint: orders must be placed during campaign or within 7 days after
        .filter(
            (col("order_placed_at") >= col("campaign_start")) &
            (col("order_placed_at") <= expr("date_add(campaign_end, 7)"))
        )
        # Prevent double counting - each order attributed once per campaign
        .dropDuplicates(["order_id", "campaign_id"])
        .groupBy("campaign_id")
        .agg(spark_sum(col("total_amount").cast(DoubleType())).alias("campaign_revenue")),
        "campaign_id",
        "left"  # LEFT JOIN to keep all campaigns even without revenue
    ) \
    .withColumn("revenue_generated", coalesce(col("campaign_revenue"), lit(0))) \
    .drop("campaign_revenue") \
    .withColumns(
        {
            "campaign_duration_days": when(
                col("start_date").isNotNull() & col("end_date").isNotNull(),
                datediff(to_date(col("end_date")), to_date(col("start_date")))
            ),
            "campaign_roi": when(
                col("spent_amount") > 0,
                ((col("revenue_generated") - col("spent_amount")) / col("spent_amount")) * 100
            ).otherwise(None),
            "click_through_rate": when(
                (col("impressions") > 0) & col("clicks").isNotNull() & col("impressions").isNotNull(),
                (col("clicks") / col("impressions")) * 100
            ).otherwise(None),
            "conversion_rate": when(
                (col("impressions") > 0) & col("conversions").isNotNull() & col("impressions").isNotNull(),
                (col("conversions") / col("impressions")) * 100
            ).otherwise(None),
            "cost_per_conversion": when(
                (col("conversions") > 0) & col("spent_amount").isNotNull(),
                col("spent_amount") / col("conversions")
            ).otherwise(None),
            "cost_per_click": when(
                (col("clicks") > 0) & col("spent_amount").isNotNull(),
                col("spent_amount") / col("clicks")
            ).otherwise(None),
            "campaign_efficiency_score": when(
                col("conversions").isNotNull() & col("budget").isNotNull() & (col("budget") != 0),
                (col("conversions") * 100 ) / col("budget")
            ).otherwise(None)
        }
    )


In [15]:
dataframes["marketing_campaigns"].show(1000)

+-----------+--------------------+-------------+----------+----------+---------+------------+-----------+------+-----------+--------------------+---------------+------------------+----------------------+--------------------+-------------------+--------------------+-------------------+--------------------+-------------------------+
|campaign_id|       campaign_name|campaign_type|start_date|  end_date|   budget|spent_amount|impressions|clicks|conversions|     target_audience|campaign_status| revenue_generated|campaign_duration_days|        campaign_roi| click_through_rate|     conversion_rate|cost_per_conversion|      cost_per_click|campaign_efficiency_score|
+-----------+--------------------+-------------+----------+----------+---------+------------+-----------+------+-----------+--------------------+---------------+------------------+----------------------+--------------------+-------------------+--------------------+-------------------+--------------------+-------------------------+
|

In [16]:
dataframes["wishlist"] = dataframes["wishlist"].withColumns(
    {
        "wishlist_to_purchase_time": when(
            col("added_date").isNotNull() & col("purchased_date").isNotNull(),
            greatest(
                unix_timestamp(to_timestamp(col("purchased_date")))
                - unix_timestamp(to_timestamp(col("added_date"))),
                lit(0),
            ),
        ),
    }
).dropDuplicates(["wishlist_id"])

In [17]:
dataframes["shopping_cart"] = dataframes["shopping_cart"].withColumn(
    "cart_age_time",
    when(
        col("added_date").isNotNull(),
        greatest(
            unix_timestamp(current_date())
            - unix_timestamp(to_timestamp(col("added_date"))),
            lit(0),
        ),
    ),
)

dataframes["shopping_cart"] = dataframes["shopping_cart"].join(
    dataframes["shopping_cart"]
    .join(dataframes["customer_sessions"], "session_id", "left")
    .join(dataframes["orders"], "customer_id", "left")
    .select(
        col("cart_id"),
        when(
            (col("products_viewed") > 0) &
            (col("order_id").isNull()) &
            (col("cart_id").isNotNull()),
            lit("Active")
        ).otherwise(lit("Inactive")).alias("cart_abandonment_flag")
    )
    .dropDuplicates(["cart_id"]),
    "cart_id",
    "left"
).dropDuplicates(["cart_id"])

In [18]:
dataframes["products"] = dataframes["products"].withColumns(
    {
        "profit_margin": when(
            col("cost_price").isNotNull() & col("sell_price").isNotNull() & (col("cost_price") != 0) & (col("sell_price") != 0),
            col("sell_price") - col("cost_price")   
        ),
    }
)

In [19]:
dataframes["inventory"].show(3)

+------------+----------+-----------+--------------+-----------------+-------------------+-------------------+------------+
|inventory_id|product_id|supplier_id|stock_quantity|reserved_quantity|minimum_stock_level|last_restocked_date|storage_cost|
+------------+----------+-----------+--------------+-----------------+-------------------+-------------------+------------+
|       10847|      1984|      10147|            82|                8|                 26|         2025-10-08|        2.64|
|       10421|      1818|      10399|            61|                6|                 68|         2025-10-24|        4.71|
|       10772|      1482|      10222|             0|                0|                 43|         2025-10-24|        4.32|
+------------+----------+-----------+--------------+-----------------+-------------------+-------------------+------------+
only showing top 3 rows



In [20]:
dataframes["inventory"] = (
    dataframes["inventory"]
    .join(total_sold_df, "product_id", "left")
    .join(avg_inventory_df, "product_id", "left")
    .withColumns(
        {
            "storage_cost_per_unit": when(
                col("storage_cost").isNotNull()
                & col("stock_quantity").isNotNull()
                & (col("stock_quantity") != 0)
                & (col("storage_cost") != 0),
                col("storage_cost") / col("stock_quantity")
            ),
            "stock_status": when(
                col("stock_quantity").isNotNull(),
                when(col("stock_quantity") == 0, "Out of Stock")
                .when(
                    (col("stock_quantity") > 0)
                    & (col("stock_quantity") <= col("reserved_quantity")),
                    "Low Stock",
                )
                .when(
                    (col("stock_quantity") > col("reserved_quantity"))
                    & (col("stock_quantity") <= 50),
                    "In Stock",
                )
                .otherwise("High Stock"),
            ),
            "available_stock": when(
                col("stock_quantity").isNotNull()
                & col("reserved_quantity").isNotNull()
                & (col("stock_quantity") >= col("reserved_quantity")),
                col("stock_quantity") - col("reserved_quantity"),
            ),
            "stock_coverage_days": when(
                col("stock_quantity").isNotNull() & (avg_daily_sales > lit(0)),
                col("stock_quantity") / avg_daily_sales,
            ),
            "reorder_point_breach": when(
                col("stock_quantity").isNotNull()
                & col("minimum_stock_level").isNotNull(),
                when(col("stock_quantity") < col("minimum_stock_level"), 1).otherwise(0),
            ),
            "stock_turnover_ratio": when(
                col("avg_inventory").isNotNull()
                & (col("avg_inventory") != 0)
                & col("total_sold").isNotNull(),
                col("total_sold") / col("avg_inventory"),
            ),
        }
    )
)

In [21]:
dataframes["customer_sessions"] = dataframes["customer_sessions"].withColumns(
    {
        "session_duration_seconds": when(
            col("session_start").isNotNull() & col("session_end").isNotNull(),
            greatest(
                unix_timestamp(to_timestamp(col("session_end")))
                - unix_timestamp(to_timestamp(col("session_start"))),
                lit(0),
            ),
        ),
        "session_duration_minutes": when(
            col("session_start").isNotNull() & col("session_end").isNotNull(),
            greatest(
                floor(
                    (
                        unix_timestamp(to_timestamp(col("session_end")))
                        - unix_timestamp(to_timestamp(col("session_start")))
                    )
                    / 60
                ),
                lit(0),
            ),
        ),
        "session_duration_hours": when(
            col("session_start").isNotNull() & col("session_end").isNotNull(),
            greatest(
                floor(
                    (
                        unix_timestamp(to_timestamp(col("session_end")))
                        - unix_timestamp(to_timestamp(col("session_start")))
                    )
                    / 3600
                ),
                lit(0),
            ),
        ),
        "pages_per_minute": when(
            col("session_duration_minutes").isNotNull()
            & (col("session_duration_minutes") != 0)
            & col("pages_viewed").isNotNull()
            & (col("pages_viewed") != 0),
            col("pages_viewed") / col("session_duration_minutes"),
        ),
        "products_per_page": when(
            col("pages_viewed").isNotNull()
            & (col("pages_viewed") != 0)
            & col("products_viewed").isNotNull()
            & (col("products_viewed") != 0),
            col("products_viewed") / col("pages_viewed"),
        ),
        "conversion_flag": when(
            col("conversion_flag").isNotNull(),
            when(
                (col("conversion_flag") == "true") | (col("conversion_flag") == "True"),
                1,
            ).otherwise(0),
        ),
        "cart_abandonment_flag": when(
            col("cart_abandonment_flag").isNotNull(),
            when(
                (col("cart_abandonment_flag") == "true")
                | (col("cart_abandonment_flag") == "True"),
                1,
            ).otherwise(0),
        ),
    }
)

In [22]:
dataframes["reviews"] = dataframes["reviews"].withColumns(
    {
        "review_sentiment": when(
            col("rating").isNotNull() & (col("rating") != 0),
            when(col("rating") >= 4, "Positive")
            .when(col("rating") == 3, "Neutral")
            .otherwise("Negative")
        )
    }
).dropDuplicates(["review_id"])

# Aggregations

## Customer

In [23]:
orders_with_items = dataframes["orders"] \
    .join(dataframes["order_items"], "order_id", "inner") \
    .select(
        "order_id",
        "customer_id",
        "order_status",
        "total_amount",
        "total_discount",
        "order_placed_at",
        "quantity"
    )

In [24]:
window_spec = Window.partitionBy("customer_id").orderBy("order_placed_at")
orders_with_lag = orders_with_items \
    .withColumn("prev_order_date", lag("order_placed_at", 1).over(window_spec)) \
    .withColumn(
        "days_since_prev_order",
        when(
            col("prev_order_date").isNotNull(),
            datediff(col("order_placed_at"), col("prev_order_date"))
        )
    )

In [25]:
customer_order_agg = orders_with_lag.groupBy("customer_id").agg(
    # Total orders
    countDistinct("order_id").alias("total_orders"),
    
    # Revenue metrics
    spark_sum(
        when(
            col("total_amount").isNotNull() & (col("total_amount") != 0),
            col("total_amount")
        )
    ).alias("total_revenue"),
    
    spark_avg(
        when(
            col("total_amount").isNotNull() & (col("total_amount") != 0),
            col("total_amount")
        )
    ).alias("avg_order_value"),
    
    # Items purchased
    spark_sum(
        when(
            col("quantity").isNotNull() & (col("quantity") > 0),
            col("quantity")
        )
    ).alias("total_items_purchased"),
    
    spark_avg(
        when(
            col("quantity").isNotNull() & (col("quantity") > 0),
            col("quantity")
        )
    ).alias("avg_items_per_order"),
    
    # Discount metrics
    spark_sum(
        when(
            col("total_discount").isNotNull() & (col("total_discount") > 0),
            col("total_discount")
        )
    ).alias("total_discount_received"),
    
    spark_avg(
        when(
            col("total_discount").isNotNull() & (col("total_discount") > 0),
            col("total_discount")
        )
    ).alias("avg_discount_per_order"),
    
    # Order dates
    spark_min(
        when(col("order_placed_at").isNotNull(), col("order_placed_at"))
    ).alias("first_order_date"),
    
    spark_max(
        when(col("order_placed_at").isNotNull(), col("order_placed_at"))
    ).alias("last_order_date"),
    
    # Days between orders
    spark_avg(
        when(
            col("days_since_prev_order").isNotNull() & (col("days_since_prev_order") > 0),
            col("days_since_prev_order")
        )
    ).alias("avg_days_between_orders"),
    
    # Cancelled orders
    spark_sum(
        when(
            col("order_status").isNotNull() & 
            (lower(col("order_status")) == "cancelled"),
            lit(1)
        ).otherwise(lit(0))
    ).alias("total_cancelled_orders")
)

In [26]:
customer_review_agg = dataframes["reviews"] \
    .filter(col("customer_id").isNotNull()) \
    .groupBy("customer_id").agg(
        count("review_id").alias("total_reviews_written"),
        spark_avg(
            when(
                col("rating").isNotNull() & (col("rating") > 0),
                col("rating")
            )
        ).alias("avg_review_rating")
    )

In [27]:
customer_session_agg = dataframes["customer_sessions"] \
    .filter(col("customer_id").isNotNull()) \
    .groupBy("customer_id").agg(
        countDistinct("session_id").alias("total_sessions"),
        
        spark_avg(
            when(
                col("session_duration_minutes").isNotNull() & 
                (col("session_duration_minutes") > 0),
                col("session_duration_minutes")
            )
        ).alias("avg_session_duration"),
        
        spark_sum(
            when(
                col("pages_viewed").isNotNull() & (col("pages_viewed") > 0),
                col("pages_viewed")
            )
        ).alias("total_pages_viewed"),
        
        spark_sum(
            when(
                col("products_viewed").isNotNull() & (col("products_viewed") > 0),
                col("products_viewed")
            )
        ).alias("total_products_viewed"),
        
        # Conversion rate: conversions / total sessions
        (spark_sum(
            when(col("conversion_flag") == 1, lit(1)).otherwise(lit(0))
        ) / count("session_id")).alias("session_conversion_rate"),
        
        # Cart abandonment rate
        (spark_sum(
            when(col("cart_abandonment_flag") == 1, lit(1)).otherwise(lit(0))
        ) / count("session_id")).alias("cart_abandonment_rate"),
        
        # Preferred device (most common)
        expr("first(device_type)").alias("preferred_device_type"),
        
        # Preferred referrer source
        expr("first(referrer_source)").alias("preferred_referrer_source")
    )

In [28]:
customer_wishlist_agg = dataframes["wishlist"] \
    .filter(col("customer_id").isNotNull()) \
    .groupBy("customer_id").agg(
        countDistinct("wishlist_id").alias("wishlist_items_count"),
        
        # Wishlist conversion rate
        (spark_sum(
            when(col("purchased_date").isNotNull(), lit(1)).otherwise(lit(0))
        ) / count("wishlist_id")).alias("wishlist_conversion_rate")
    )

In [29]:
customer_payment_agg = dataframes["payments"] \
    .join(dataframes["orders"].select("order_id", "customer_id"), "order_id", "inner") \
    .filter(col("customer_id").isNotNull() & col("payment_method").isNotNull()) \
    .groupBy("customer_id").agg(
        expr("first(payment_method)").alias("preferred_payment_method")
    )

In [30]:
dataframes["customers"] = dataframes["customers"] \
    .join(customer_order_agg, "customer_id", "left") \
    .join(customer_review_agg, "customer_id", "left") \
    .join(customer_session_agg, "customer_id", "left") \
    .join(customer_wishlist_agg, "customer_id", "left") \
    .join(customer_payment_agg, "customer_id", "left")

In [31]:
dataframes["customers"] = dataframes["customers"].withColumns({
    # Customer lifetime value (already calculated as total_revenue)
    "customer_lifetime_value": coalesce(col("total_revenue"), lit(0)),
    
    # Days since last purchase
    "days_since_last_purchase": when(
        col("last_order_date").isNotNull(),
        datediff(current_date(), col("last_order_date"))
    ),
    
    # Is repeat customer
    "is_repeat_customer": when(
        col("total_orders").isNotNull() & (col("total_orders") > 1),
        lit(1)
    ).otherwise(lit(0)),
    
    # Cancellation rate
    "cancellation_rate": when(
        col("total_orders").isNotNull() & 
        (col("total_orders") > 0) & 
        col("total_cancelled_orders").isNotNull(),
        (col("total_cancelled_orders") / col("total_orders")) * 100
    ),
    
    # Customer activity score (weighted: recency=40%, frequency=40%, monetary=20%)
    "customer_activity_score": when(
        col("days_since_last_purchase").isNotNull() &
        col("total_orders").isNotNull() &
        col("total_revenue").isNotNull(),
        (
            # Recency score (inverse - lower days is better)
            (lit(1) / (col("days_since_last_purchase") + lit(1))) * lit(0.4) +
            # Frequency score
            (col("total_orders") / lit(100)) * lit(0.4) +
            # Monetary score
            (col("total_revenue") / lit(10000)) * lit(0.2)
        ) * lit(100)
    ),
    
    # Handle null values with defaults
    "total_orders": coalesce(col("total_orders"), lit(0)),
    "total_revenue": coalesce(col("total_revenue"), lit(0.0)),
    "total_items_purchased": coalesce(col("total_items_purchased"), lit(0)),
    "total_reviews_written": coalesce(col("total_reviews_written"), lit(0)),
    "total_sessions": coalesce(col("total_sessions"), lit(0)),
    "wishlist_items_count": coalesce(col("wishlist_items_count"), lit(0)),
})

## Product

In [32]:
order_items_enhanced = dataframes["order_items"] \
    .join(
        dataframes["products"].select("product_id", "sell_price", "cost_price"),
        "product_id",
        "left"
    ) \
    .select(
        "order_item_id",
        "order_id",
        "product_id",
        "quantity",
        "discount_amount",
        "product_cost",
        "sell_price",
        "cost_price"
    )

In [33]:
product_sales_agg = order_items_enhanced \
    .filter(col("product_id").isNotNull()) \
    .join(dataframes["orders"].select("order_id", "customer_id"), "order_id", "inner") \
    .groupBy("product_id").agg(
        # Total units sold
        spark_sum(
            when(
                col("quantity").isNotNull() & (col("quantity") > 0),
                col("quantity")
            )
        ).alias("total_units_sold"),
        
        # Total revenue (quantity * sell_price)
        spark_sum(
            when(
                col("quantity").isNotNull() & 
                col("sell_price").isNotNull() &
                (col("quantity") > 0) & 
                (col("sell_price") > 0),
                col("quantity") * col("sell_price")
            )
        ).alias("total_revenue"),
        
        # Total profit ((sell_price - cost_price) * quantity)
        spark_sum(
            when(
                col("quantity").isNotNull() & 
                col("sell_price").isNotNull() &
                col("cost_price").isNotNull() &
                (col("quantity") > 0) & 
                (col("sell_price") > 0) &
                (col("cost_price") >= 0),
                (col("sell_price") - col("cost_price")) * col("quantity")
            )
        ).alias("total_profit"),
        
        # Average profit margin
        spark_avg(
            when(
                col("sell_price").isNotNull() &
                col("cost_price").isNotNull() &
                (col("sell_price") > 0) &
                (col("cost_price") >= 0),
                col("sell_price") - col("cost_price")
            )
        ).alias("avg_profit_margin"),
        
        # Total orders
        countDistinct(
            when(col("order_id").isNotNull(), col("order_id"))
        ).alias("total_orders"),
        
        # Unique customers
        countDistinct(
            when(col("customer_id").isNotNull(), col("customer_id"))
        ).alias("unique_customers"),
        
        # Average quantity per order
        spark_avg(
            when(
                col("quantity").isNotNull() & (col("quantity") > 0),
                col("quantity")
            )
        ).alias("avg_quantity_per_order"),
        
        # Average discount amount
        spark_avg(
            when(
                col("discount_amount").isNotNull() & (col("discount_amount") > 0),
                col("discount_amount")
            )
        ).alias("avg_discount_amount")
    )

In [34]:
product_review_agg = dataframes["reviews"] \
    .filter(col("product_id").isNotNull()) \
    .groupBy("product_id").agg(
        # Total reviews
        count("review_id").alias("total_reviews"),
        
        # Average rating
        spark_avg(
            when(
                col("rating").isNotNull() & (col("rating") > 0),
                col("rating")
            )
        ).alias("avg_rating"),
        
        # Rating standard deviation
        expr("stddev(CASE WHEN rating IS NOT NULL AND rating > 0 THEN rating ELSE NULL END)").alias("rating_std_dev"),
        
        # Positive review rate (rating >= 4)
        (spark_sum(
            when(
                col("rating").isNotNull() & (col("rating") >= 4),
                lit(1)
            ).otherwise(lit(0))
        ) / count("review_id")).alias("positive_review_rate")
    )

In [35]:
product_wishlist_agg = dataframes["wishlist"] \
    .filter(col("product_id").isNotNull()) \
    .groupBy("product_id").agg(
        # Total wishlist adds
        count("wishlist_id").alias("total_wishlist_adds"),
        
        # Wishlist to purchase rate
        (spark_sum(
            when(col("purchased_date").isNotNull(), lit(1)).otherwise(lit(0))
        ) / count("wishlist_id")).alias("wishlist_to_purchase_rate")
    )

In [36]:
product_cart_agg = dataframes["shopping_cart"] \
    .filter(col("product_id").isNotNull()) \
    .groupBy("product_id").agg(
        # Total cart adds
        count("cart_id").alias("total_cart_adds"),
        
        # Cart to purchase rate (cart_status = 'purchased' or similar indicator)
        (spark_sum(
            when(
                col("cart_status").isNotNull() & 
                (lower(col("cart_status")).isin("purchased", "completed", "ordered")),
                lit(1)
            ).otherwise(lit(0))
        ) / count("cart_id")).alias("cart_to_purchase_rate")
    )

In [37]:
product_view_agg = dataframes["customer_sessions"] \
    .filter(
        col("products_viewed").isNotNull() & 
        (col("products_viewed") > 0)
    ) \
    .groupBy().agg(
        spark_sum("products_viewed").alias("total_views_all_products")
    ).collect()[0]["total_views_all_products"]

In [38]:
product_inventory_agg = dataframes["inventory"] \
    .filter(col("product_id").isNotNull()) \
    .groupBy("product_id").agg(
        # Current stock level (latest/last value)
        expr("last(stock_quantity)").alias("current_stock_level"),
        
        # Stockout days calculation
        spark_sum(
            when(
                col("stock_quantity").isNotNull() & (col("stock_quantity") == 0),
                lit(1)
            ).otherwise(lit(0))
        ).alias("stockout_occurrences")
    )

In [39]:
inventory_restock = dataframes["inventory"] \
    .filter(
        col("product_id").isNotNull() & 
        col("last_restocked_date").isNotNull()
    ) \
    .select("product_id", "last_restocked_date") \
    .distinct()

window_restock = Window.partitionBy("product_id").orderBy("last_restocked_date")

inventory_with_prev = inventory_restock \
    .withColumn("prev_restock_date", lag("last_restocked_date", 1).over(window_restock)) \
    .withColumn(
        "days_between_restocks",
        when(
            col("prev_restock_date").isNotNull(),
            datediff(col("last_restocked_date"), col("prev_restock_date"))
        )
    )

product_restock_agg = inventory_with_prev \
    .filter(col("days_between_restocks").isNotNull()) \
    .groupBy("product_id").agg(
        spark_avg("days_between_restocks").alias("avg_restock_frequency")
    )

In [40]:
dataframes["products"] = dataframes["products"] \
    .join(product_sales_agg, "product_id", "left") \
    .join(product_review_agg, "product_id", "left") \
    .join(product_wishlist_agg, "product_id", "left") \
    .join(product_cart_agg, "product_id", "left") \
    .join(product_inventory_agg, "product_id", "left") \
    .join(product_restock_agg, "product_id", "left")

In [41]:
dataframes["products"] = dataframes["products"].withColumns({
    # View to purchase rate (if view data is available per product)
    # Note: Adjust if you have product-specific view counts
    "view_to_purchase_rate": when(
        col("total_units_sold").isNotNull() &
        (col("total_units_sold") > 0),
        col("total_units_sold") / lit(100)  # Placeholder - adjust with actual view data
    ),
    
    # Revenue per view (if view data is available)
    "revenue_per_view": when(
        col("total_revenue").isNotNull() &
        (col("total_revenue") > 0),
        col("total_revenue") / lit(100)  # Placeholder - adjust with actual view data
    ),
    
    # Days since launch
    "days_since_launch": when(
        col("launch_date").isNotNull(),
        datediff(current_date(), col("launch_date"))
    ),
    
    # Stockout days (approximation based on occurrences)
    "stockout_days": coalesce(col("stockout_occurrences"), lit(0)),
    
    # Product performance score (weighted metric)
    "product_performance_score": when(
        col("total_units_sold").isNotNull() &
        col("avg_rating").isNotNull() &
        col("total_profit").isNotNull(),
        (
            # Sales volume (40%)
            (col("total_units_sold") / lit(1000)) * lit(0.4) +
            # Rating (30%)
            (col("avg_rating") / lit(5)) * lit(0.3) +
            # Profitability (30%)
            (col("total_profit") / lit(10000)) * lit(0.3)
        ) * lit(100)
    ),
    
    # Inventory turnover rate
    "inventory_turnover_rate": when(
        col("current_stock_level").isNotNull() &
        (col("current_stock_level") > 0) &
        col("total_units_sold").isNotNull(),
        col("total_units_sold") / col("current_stock_level")
    ),
    
    # Average order value for this product
    "avg_order_value_product": when(
        col("total_revenue").isNotNull() &
        col("total_orders").isNotNull() &
        (col("total_orders") > 0),
        col("total_revenue") / col("total_orders")
    ),
    
    # Customer penetration rate
    "customer_penetration": when(
        col("unique_customers").isNotNull() &
        (col("unique_customers") > 0),
        col("unique_customers") / lit(2000)  # Adjust denominator to total customer count
    ),
    
    # Handle null values with defaults
    "total_units_sold": coalesce(col("total_units_sold"), lit(0)),
    "total_revenue": coalesce(col("total_revenue"), lit(0.0)),
    "total_profit": coalesce(col("total_profit"), lit(0.0)),
    "total_orders": coalesce(col("total_orders"), lit(0)),
    "unique_customers": coalesce(col("unique_customers"), lit(0)),
    "total_reviews": coalesce(col("total_reviews"), lit(0)),
    "avg_rating": coalesce(col("avg_rating"), lit(0.0)),
    "total_wishlist_adds": coalesce(col("total_wishlist_adds"), lit(0)),
    "total_cart_adds": coalesce(col("total_cart_adds"), lit(0)),
    "current_stock_level": coalesce(col("current_stock_level"), lit(0)),
    "stockout_occurrences": coalesce(col("stockout_occurrences"), lit(0)),
})

In [42]:
product_category_performance = dataframes["products"] \
    .filter(col("category").isNotNull()) \
    .groupBy("category").agg(
        spark_sum("total_revenue").alias("category_total_revenue"),
        # spark_avg("avg_rating").alias("category_avg_rating"),
        count("product_id").alias("products_in_category")
    )

dataframes["products"] = dataframes["products"] \
    .join(product_category_performance, "category", "left") \
    .withColumn(
        "product_category_revenue_share",
        when(
            col("total_revenue").isNotNull() &
            col("category_total_revenue").isNotNull() &
            (col("category_total_revenue") > 0),
            (col("total_revenue") / col("category_total_revenue")) * 100
        )
    )

## Category

In [43]:
order_items_with_category = dataframes["order_items"] \
    .join(
        dataframes["products"].select("product_id", "category", "sell_price"),
        "product_id",
        "left"
    ) \
    .join(
        dataframes["orders"].select("order_id", "customer_id", "order_placed_at"),
        "order_id",
        "inner"
    ) \
    .select(
        "order_item_id",
        "order_id",
        "product_id",
        "category",
        "quantity",
        "product_cost",
        "sell_price",
        "customer_id",
        "order_placed_at"
    )

In [44]:
category_agg = order_items_with_category \
    .filter(col("category").isNotNull()) \
    .groupBy("category").agg(
        # Total products in category
        countDistinct(
            when(col("product_id").isNotNull(), col("product_id"))
        ).alias("total_products_in_category"),
        
        # Total revenue (quantity * sell_price)
        spark_sum(
            when(
                col("quantity").isNotNull() & 
                col("sell_price").isNotNull() &
                (col("quantity") > 0) & 
                (col("sell_price") > 0),
                col("quantity") * col("sell_price")
            )
        ).alias("total_revenue"),
        
        # Total units sold
        spark_sum(
            when(
                col("quantity").isNotNull() & (col("quantity") > 0),
                col("quantity")
            )
        ).alias("total_units_sold"),
        
        # Total orders
        countDistinct(
            when(col("order_id").isNotNull(), col("order_id"))
        ).alias("total_orders"),
        
        # Unique customers
        countDistinct(
            when(col("customer_id").isNotNull(), col("customer_id"))
        ).alias("unique_customers"),
        
        # Average items per order
        spark_avg(
            when(
                col("quantity").isNotNull() & (col("quantity") > 0),
                col("quantity")
            )
        ).alias("avg_items_per_order")
    )

In [45]:
category_price_agg = dataframes["products"] \
    .filter(col("category").isNotNull()) \
    .groupBy("category").agg(
        # Average product price in category
        spark_avg(
            when(
                col("sell_price").isNotNull() & (col("sell_price") > 0),
                col("sell_price")
            )
        ).alias("avg_product_price")
    )

In [46]:
category_rating_agg = dataframes["reviews"] \
    .join(
        dataframes["products"].select("product_id", "category"),
        "product_id",
        "inner"
    ) \
    .filter(col("category").isNotNull()) \
    .groupBy("category").agg(
        # Average rating
        spark_avg(
            when(
                col("rating").isNotNull() & (col("rating") > 0),
                col("rating")
            )
        ).alias("avg_rating"),
        
        # Total reviews
        count("review_id").alias("total_reviews")
    )

In [47]:
total_revenue_all = order_items_with_category \
    .filter(
        col("quantity").isNotNull() & 
        col("sell_price").isNotNull() &
        (col("quantity") > 0) & 
        (col("sell_price") > 0)
    ) \
    .agg(
        spark_sum(col("quantity") * col("sell_price")).alias("grand_total_revenue")
    ).collect()[0]["grand_total_revenue"]

In [48]:
order_items_with_month = order_items_with_category \
    .filter(
        col("order_placed_at").isNotNull() &
        col("category").isNotNull()
    ) \
    .withColumn("order_year", year(col("order_placed_at"))) \
    .withColumn("order_month", month(col("order_placed_at"))) \
    .withColumn(
        "year_month",
        expr("concat(order_year, '-', lpad(order_month, 2, '0'))")
    )

In [49]:
category_monthly_revenue = order_items_with_month \
    .groupBy("category", "year_month", "order_year", "order_month").agg(
        spark_sum(
            when(
                col("quantity").isNotNull() & 
                col("sell_price").isNotNull() &
                (col("quantity") > 0) & 
                (col("sell_price") > 0),
                col("quantity") * col("sell_price")
            )
        ).alias("monthly_revenue")
    )

In [50]:
window_growth = Window.partitionBy("category").orderBy("order_year", "order_month")

category_growth = category_monthly_revenue \
    .withColumn("prev_month_revenue", lag("monthly_revenue", 1).over(window_growth)) \
    .withColumn(
        "month_over_month_growth",
        when(
            col("prev_month_revenue").isNotNull() & 
            (col("prev_month_revenue") > 0),
            ((col("monthly_revenue") - col("prev_month_revenue")) / col("prev_month_revenue")) * 100
        )
    )

# Average growth rate per category
category_growth_agg = category_growth \
    .filter(col("month_over_month_growth").isNotNull()) \
    .groupBy("category").agg(
        spark_avg("month_over_month_growth").alias("avg_category_growth_rate")
    )

In [51]:
order_items_with_season = order_items_with_category \
    .filter(
        col("order_placed_at").isNotNull() &
        col("category").isNotNull()
    ) \
    .withColumn("order_month", month(col("order_placed_at"))) \
    .withColumn(
        "season",
        when(col("order_month").isin(12, 1, 2), "Winter")
        .when(col("order_month").isin(3, 4, 5), "Spring")
        .when(col("order_month").isin(6, 7, 8), "Summer")
        .otherwise("Fall")
    )

In [52]:
category_seasonal_revenue = order_items_with_season \
    .groupBy("category", "season").agg(
        spark_sum(
            when(
                col("quantity").isNotNull() & 
                col("sell_price").isNotNull() &
                (col("quantity") > 0) & 
                (col("sell_price") > 0),
                col("quantity") * col("sell_price")
            )
        ).alias("seasonal_revenue")
    )


In [53]:
category_avg_seasonal = category_seasonal_revenue \
    .groupBy("category").agg(
        spark_avg("seasonal_revenue").alias("avg_seasonal_revenue")
    )

In [54]:
category_seasonal_with_avg = category_seasonal_revenue \
    .join(category_avg_seasonal, "category", "inner") \
    .withColumn(
        "seasonal_index",
        when(
            col("avg_seasonal_revenue").isNotNull() & 
            (col("avg_seasonal_revenue") > 0),
            (col("seasonal_revenue") / col("avg_seasonal_revenue")) * 100
        )
    )

In [55]:
category_seasonal_pivot = category_seasonal_with_avg \
    .groupBy("category").pivot("season").agg(
        expr("first(seasonal_index)")
    ) \
    .withColumnRenamed("Winter", "seasonal_index_winter") \
    .withColumnRenamed("Spring", "seasonal_index_spring") \
    .withColumnRenamed("Summer", "seasonal_index_summer") \
    .withColumnRenamed("Fall", "seasonal_index_fall")

In [56]:
categories_df = dataframes["products"] \
    .select("category") \
    .filter(col("category").isNotNull()) \
    .distinct()

In [57]:
categories_df = categories_df \
    .join(category_agg, "category", "left") \
    .join(category_price_agg, "category", "left") \
    .join(category_rating_agg, "category", "left") \
    .join(category_growth_agg, "category", "left") \
    .join(category_seasonal_pivot, "category", "left")

In [58]:


# Check which columns exist
available_columns = categories_df.columns

# Define the transformations dictionary
transformations = {
    # Revenue share percentage
    "revenue_share_percentage": when(
        col("total_revenue").isNotNull() &
        (col("total_revenue") > 0) &
        (lit(total_revenue_all) > 0),
        (col("total_revenue") / lit(total_revenue_all)) * 100
    ),
    
    # Average order value for category
    "avg_order_value": when(
        col("total_revenue").isNotNull() &
        col("total_orders").isNotNull() &
        (col("total_orders") > 0),
        col("total_revenue") / col("total_orders")
    ),
    
    # Revenue per customer
    "revenue_per_customer": when(
        col("total_revenue").isNotNull() &
        col("unique_customers").isNotNull() &
        (col("unique_customers") > 0),
        col("total_revenue") / col("unique_customers")
    ),
    
    # Average units per customer
    "avg_units_per_customer": when(
        col("total_units_sold").isNotNull() &
        col("unique_customers").isNotNull() &
        (col("unique_customers") > 0),
        col("total_units_sold") / col("unique_customers")
    ),
    
    # Category popularity score (normalized)
    "category_popularity_score": when(
        col("total_orders").isNotNull() &
        col("unique_customers").isNotNull() &
        col("avg_rating").isNotNull(),
        (
            (col("total_orders") / lit(1000)) * lit(0.4) +
            (col("unique_customers") / lit(500)) * lit(0.3) +
            (col("avg_rating") / lit(5)) * lit(0.3)
        ) * lit(100)
    ),
    
    # Product diversity index
    "product_diversity_index": when(
        col("total_products_in_category").isNotNull() &
        col("total_units_sold").isNotNull() &
        (col("total_products_in_category") > 0),
        col("total_units_sold") / col("total_products_in_category")
    ),
    
    # Orders per product
    "avg_orders_per_product": when(
        col("total_orders").isNotNull() &
        col("total_products_in_category").isNotNull() &
        (col("total_products_in_category") > 0),
        col("total_orders") / col("total_products_in_category")
    ),
    
    # Handle null values with defaults
    "total_products_in_category": coalesce(col("total_products_in_category"), lit(0)),
    "total_revenue": coalesce(col("total_revenue"), lit(0.0)),
    "total_units_sold": coalesce(col("total_units_sold"), lit(0)),
    "total_orders": coalesce(col("total_orders"), lit(0)),
    "unique_customers": coalesce(col("unique_customers"), lit(0)),
    "total_reviews": coalesce(col("total_reviews"), lit(0)),
    "avg_rating": coalesce(col("avg_rating"), lit(0.0)),
    "avg_category_growth_rate": coalesce(col("avg_category_growth_rate"), lit(0.0)),
}

# Only add peak_season if all seasonal columns exist
seasonal_columns = ["seasonal_index_winter", "seasonal_index_spring", 
                   "seasonal_index_summer", "seasonal_index_fall"]

if all(col_name in available_columns for col_name in seasonal_columns):
    transformations["peak_season"] = greatest(
        coalesce(col("seasonal_index_winter"), lit(0)),
        coalesce(col("seasonal_index_spring"), lit(0)),
        coalesce(col("seasonal_index_summer"), lit(0)),
        coalesce(col("seasonal_index_fall"), lit(0))
    )
else:
    # Set to null if seasonal data not available
    transformations["peak_season"] = lit(None).cast("double")

# Apply all transformations
categories_df = categories_df.withColumns(transformations)

In [59]:
dataframes["categories"] = categories_df

# Optional: Add category rankings
categories_with_rank = categories_df \
    .withColumn(
        "revenue_rank",
        expr("row_number() OVER (ORDER BY total_revenue DESC)")
    ) \
    .withColumn(
        "rating_rank",
        expr("row_number() OVER (ORDER BY avg_rating DESC)")
    ) \
    .withColumn(
        "growth_rank",
        expr("row_number() OVER (ORDER BY avg_category_growth_rate DESC)")
    )

dataframes["categories"] = categories_with_rank

# Also add category performance back to products for easy reference
dataframes["products"] = dataframes["products"] \
    .drop("category_performance_tier") \
    .join(
        categories_df.select(
            "category",
            "revenue_share_percentage",
            "avg_category_growth_rate",
            col("avg_rating").alias("category_avg_rating")
        ),
        "category",
        "left"
    ) \
    .withColumn(
        "category_performance_tier",
        when(
            col("revenue_share_percentage").isNotNull(),
            when(col("revenue_share_percentage") >= 20, "Top Tier")
            .when(col("revenue_share_percentage") >= 10, "Mid Tier")
            .when(col("revenue_share_percentage") >= 5, "Growing")
            .otherwise("Niche")
        )
    )

## Supplier

In [60]:
products_with_supplier = dataframes["products"].select(
    "product_id", "supplier_id", "cost_price", "sell_price"
)

order_items_with_supplier = (
    dataframes["order_items"]
    .join(products_with_supplier, "product_id", "left")
    .join(dataframes["orders"].select("order_id", "order_status"), "order_id", "inner")
)

In [61]:
supplier_sales_agg = (
    order_items_with_supplier.filter(col("supplier_id").isNotNull())
    .groupBy("supplier_id")
    .agg(
        # Total products supplied
        countDistinct(when(col("product_id").isNotNull(), col("product_id"))).alias(
            "total_products_supplied"
        ),
        # Total revenue generated (quantity * sell_price)
        spark_sum(
            when(
                col("quantity").isNotNull()
                & col("sell_price").isNotNull()
                & (col("quantity") > 0)
                & (col("sell_price") > 0),
                col("quantity") * col("sell_price"),
            )
        ).alias("total_revenue_generated"),
        # Total units sold
        spark_sum(
            when(col("quantity").isNotNull() & (col("quantity") > 0), col("quantity"))
        ).alias("total_units_sold"),
        # Total orders fulfilled
        countDistinct(when(col("order_id").isNotNull(), col("order_id"))).alias(
            "total_orders_fulfilled"
        ),
        # Average profit margin
        spark_avg(
            when(
                col("sell_price").isNotNull()
                & col("cost_price").isNotNull()
                & (col("sell_price") > 0)
                & (col("cost_price") >= 0),
                col("sell_price") - col("cost_price"),
            )
        ).alias("avg_profit_margin"),
    )
)

In [62]:
supplier_rating_agg = (
    dataframes["reviews"]
    .join(
        dataframes["products"].select("product_id", "supplier_id"),
        "product_id",
        "inner",
    )
    .filter(col("supplier_id").isNotNull())
    .groupBy("supplier_id")
    .agg(
        # Average product rating
        spark_avg(
            when(col("rating").isNotNull() & (col("rating") > 0), col("rating"))
        ).alias("avg_product_rating"),
        # Total reviews
        count("review_id").alias("total_reviews"),
    )
)

In [63]:
supplier_inventory_agg = (
    dataframes["inventory"]
    .join(
        dataframes["products"].select("product_id", col("supplier_id").alias("prod_supplier_id"), "cost_price"),
        "product_id",
        "left",
    )
    .filter(col("prod_supplier_id").isNotNull())
    .groupBy("prod_supplier_id")
    .agg(
        # Total stock value (stock_quantity * cost_price)
        spark_sum(
            when(
                col("stock_quantity").isNotNull()
                & col("cost_price").isNotNull()
                & (col("stock_quantity") > 0)
                & (col("cost_price") > 0),
                col("stock_quantity") * col("cost_price"),
            )
        ).alias("total_stock_value"),
        # Average stock quantity
        spark_avg(
            when(
                col("stock_quantity").isNotNull() & (col("stock_quantity") >= 0),
                col("stock_quantity"),
            )
        ).alias("avg_stock_quantity"),
    )
    .withColumnRenamed("prod_supplier_id", "supplier_id")
)

In [64]:
inventory_restock_supplier = (
    dataframes["inventory"]
    .select("product_id", "last_restocked_date")
    .join(
        dataframes["products"].select("product_id", "supplier_id"), "product_id", "left"
    )
    .filter(col("supplier_id").isNotNull() & col("last_restocked_date").isNotNull())
    .select("supplier_id", "product_id", "last_restocked_date")
)

In [65]:
window_supplier_restock = Window.partitionBy("supplier_id", "product_id").orderBy(
    "last_restocked_date"
)

supplier_restock_lead = (
    inventory_restock_supplier.withColumn(
        "prev_restock_date", lag("last_restocked_date", 1).over(window_supplier_restock)
    )
    .withColumn(
        "restock_lead_time",
        when(
            col("prev_restock_date").isNotNull(),
            datediff(col("last_restocked_date"), col("prev_restock_date")),
        ),
    )
    .filter(col("restock_lead_time").isNotNull())
    .groupBy("supplier_id")
    .agg(spark_avg("restock_lead_time").alias("avg_restock_lead_time"))
)

In [66]:
dataframes["suppliers"] = (
    dataframes["suppliers"]
    .join(supplier_sales_agg, "supplier_id", "left")
    .join(supplier_rating_agg, "supplier_id", "left")
    .join(supplier_inventory_agg, "supplier_id", "left")
    .join(supplier_restock_lead, "supplier_id", "left")
)

In [67]:
dataframes["suppliers"] = dataframes["suppliers"].withColumns(
    {
        # Contract status flag
        "contract_status_flag": when(
            col("contract_end_date").isNotNull(),
            when(col("contract_end_date") >= current_date(), "Active").otherwise(
                "Expired"
            ),
        ).otherwise("Unknown"),
        # Days until contract expiry
        "days_until_contract_expiry": when(
            col("contract_end_date").isNotNull(),
            datediff(col("contract_end_date"), current_date()),
        ),
        # Contract duration in days
        "contract_duration_days": when(
            col("contract_start_date").isNotNull()
            & col("contract_end_date").isNotNull(),
            datediff(col("contract_end_date"), col("contract_start_date")),
        ),
        # Revenue per product
        "revenue_per_product": when(
            col("total_revenue_generated").isNotNull()
            & col("total_products_supplied").isNotNull()
            & (col("total_products_supplied") > 0),
            col("total_revenue_generated") / col("total_products_supplied"),
        ),
        # Average order value
        "avg_order_value": when(
            col("total_revenue_generated").isNotNull()
            & col("total_orders_fulfilled").isNotNull()
            & (col("total_orders_fulfilled") > 0),
            col("total_revenue_generated") / col("total_orders_fulfilled"),
        ),
        # Units per product
        "avg_units_per_product": when(
            col("total_units_sold").isNotNull()
            & col("total_products_supplied").isNotNull()
            & (col("total_products_supplied") > 0),
            col("total_units_sold") / col("total_products_supplied"),
        ),
        # Supplier performance score
        "supplier_performance_score": when(
            col("supplier_rating").isNotNull()
            & col("avg_product_rating").isNotNull()
            & col("total_revenue_generated").isNotNull(),
            (
                # Supplier rating (40%)
                (col("supplier_rating") / lit(5)) * lit(0.4)
                +
                # Product rating (30%)
                (col("avg_product_rating") / lit(5)) * lit(0.3)
                +
                # Revenue contribution (30%)
                (col("total_revenue_generated") / lit(100000)) * lit(0.3)
            )
            * lit(100),
        ),
        # Stock efficiency ratio
        "stock_efficiency_ratio": when(
            col("total_stock_value").isNotNull()
            & col("total_revenue_generated").isNotNull()
            & (col("total_stock_value") > 0),
            (col("total_revenue_generated") / col("total_stock_value")) * 100,
        ),
        # Supplier reliability score (based on ratings and status)
        "supplier_reliability_score": when(
            col("supplier_rating").isNotNull() & (col("is_verified") == True) & (col("is_preferred") == True),
            col("supplier_rating") * lit(1.2),
        )
        .when(
            col("supplier_rating").isNotNull() & (col("is_verified") == True),
            col("supplier_rating") * lit(1.1),
        )
        .otherwise(col("supplier_rating")),
        # Handle null values with defaults
        "total_products_supplied": coalesce(col("total_products_supplied"), lit(0)),
        "total_revenue_generated": coalesce(col("total_revenue_generated"), lit(0.0)),
        "total_units_sold": coalesce(col("total_units_sold"), lit(0)),
        "total_orders_fulfilled": coalesce(col("total_orders_fulfilled"), lit(0)),
        "total_reviews": coalesce(col("total_reviews"), lit(0)),
        "avg_product_rating": coalesce(col("avg_product_rating"), lit(0.0)),
        "total_stock_value": coalesce(col("total_stock_value"), lit(0.0)),
    }
)

## Campaigns

In [68]:
from pyspark.sql.functions import (
    col,
    coalesce,
    lit,
    when,
    sum as spark_sum,
    countDistinct,
    expr,
    datediff,
    current_date,
)

# 1. Build campaign_revenue as before
campaign_revenue = (
    dataframes["customer_sessions"]
    .join(
        dataframes["marketing_campaigns"],
        (col("session_start") >= col("start_date"))
        & (col("session_start") <= col("end_date")),
        "cross",
    )
    .filter(col("conversion_flag") == 1)
    .join(
        dataframes["orders"].filter(
            (col("order_status") != "cancelled") & (col("order_status") != "refunded")
        ),
        "customer_id",
        "inner",
    )
    .filter(
        (col("order_placed_at") >= col("start_date"))
        & (col("order_placed_at") <= expr("date_add(end_date, 7)"))
    )
    .dropDuplicates(["order_id", "campaign_id"])
    .groupBy("campaign_id")
    .agg(
        spark_sum(
            when(
                col("total_amount").isNotNull() & (col("total_amount") > 0),
                col("total_amount"),
            )
        ).alias("revenue_generated"),
        countDistinct("order_id").alias("orders_from_campaign"),
    )
)

# 2. Rename aggregated columns to avoid name collision
campaign_revenue = campaign_revenue.select(
    "campaign_id",
    col("revenue_generated").alias("agg_revenue_generated"),
    col("orders_from_campaign").alias("agg_orders_from_campaign"),
)

# 3. CLEAN base marketing_campaigns: remove any existing revenue_generated* columns
mc = dataframes["marketing_campaigns"]

cols_to_drop = [c for c in mc.columns if "revenue_generated" in c]
if cols_to_drop:
    mc = mc.drop(*cols_to_drop)

# (Optional debug)
# print("After dropping:", [c for c in mc.columns if "revenue_generated" in c])

# 4. Join clean mc with campaign_revenue and define a single revenue_generated + orders_from_campaign
dataframes["marketing_campaigns"] = (
    mc.join(campaign_revenue, "campaign_id", "left")
      .withColumn(
          "revenue_generated",
          coalesce(col("agg_revenue_generated"), lit(0.0)),
      )
      .withColumn(
          "orders_from_campaign",
          coalesce(col("agg_orders_from_campaign"), lit(0)),
      )
      .drop("agg_revenue_generated", "agg_orders_from_campaign")
)

# Sanity check (you should now see exactly one revenue_generated)
# print([c for c in dataframes["marketing_campaigns"].columns if "revenue_generated" in c])

# 5. Derived metrics on marketing_campaigns
dataframes["marketing_campaigns"] = dataframes["marketing_campaigns"].withColumns({
    # Total impressions (already exists, ensure not null)
    "total_impressions": coalesce(col("impressions"), lit(0)),

    # Total clicks (already exists, ensure not null)
    "total_clicks": coalesce(col("clicks"), lit(0)),

    # Total conversions (already exists, ensure not null)
    "total_conversions": coalesce(col("conversions"), lit(0)),

    # Total budget (already exists)
    "total_budget": coalesce(col("budget"), lit(0.0)),

    # Total spent (already exists)
    "total_spent": coalesce(col("spent_amount"), lit(0.0)),

    # Budget utilization rate
    "budget_utilization_rate": when(
        col("budget").isNotNull()
        & col("spent_amount").isNotNull()
        & (col("budget") > 0),
        (col("spent_amount") / col("budget")) * 100,
    ),

    # Click-through rate (CTR)
    "ctr": when(
        col("impressions").isNotNull()
        & col("clicks").isNotNull()
        & (col("impressions") > 0),
        (col("clicks") / col("impressions")) * 100,
    ),

    # Conversion rate
    "conversion_rate": when(
        col("clicks").isNotNull()
        & col("conversions").isNotNull()
        & (col("clicks") > 0),
        (col("conversions") / col("clicks")) * 100,
    ),

    # Cost per click (CPC)
    "cost_per_click": when(
        col("spent_amount").isNotNull()
        & col("clicks").isNotNull()
        & (col("clicks") > 0),
        col("spent_amount") / col("clicks"),
    ),

    # Cost per conversion (CPA)
    "cost_per_conversion": when(
        col("conversions").isNotNull()
        & col("spent_amount").isNotNull()
        & (col("conversions") > 0),
        col("spent_amount") / col("conversions"),
    ),

    # Return on Investment (ROI)
    "roi": when(
        (col("revenue_generated") > 0)
        & col("spent_amount").isNotNull()
        & (col("spent_amount") > 0),
        ((col("revenue_generated") - col("spent_amount")) / col("spent_amount")) * 100,
    ),

    # Return on Ad Spend (ROAS)
    "roas": when(
        (col("revenue_generated") > 0)
        & col("spent_amount").isNotNull()
        & (col("spent_amount") > 0),
        col("revenue_generated") / col("spent_amount"),
    ),

    # Average order value from campaign
    "avg_order_value": when(
        (col("revenue_generated") > 0)
        & col("conversions").isNotNull()
        & (col("conversions") > 0),
        col("revenue_generated") / col("conversions"),
    ),

    # Days active (campaign duration)
    "days_active": when(
        col("start_date").isNotNull() & col("end_date").isNotNull(),
        datediff(col("end_date"), col("start_date")) + lit(1),
    ),

    # Revenue per impression
    "revenue_per_impression": when(
        (col("revenue_generated") > 0)
        & col("impressions").isNotNull()
        & (col("impressions") > 0),
        col("revenue_generated") / col("impressions"),
    ),

    # Revenue per click
    "revenue_per_click": when(
        (col("revenue_generated") > 0)
        & col("clicks").isNotNull()
        & (col("clicks") > 0),
        col("revenue_generated") / col("clicks"),
    ),

    # Profit from campaign
    "campaign_profit": when(
        col("spent_amount").isNotNull(),
        col("revenue_generated") - col("spent_amount"),
    ),

    # Cost efficiency ratio
    "cost_efficiency_ratio": when(
        (col("revenue_generated") > 0) & col("spent_amount").isNotNull(),
        (col("spent_amount") / col("revenue_generated")) * 100,
    ),

    # Engagement rate
    "engagement_rate": when(
        col("impressions").isNotNull()
        & col("clicks").isNotNull()
        & col("conversions").isNotNull()
        & (col("impressions") > 0),
        ((col("clicks") + col("conversions")) / col("impressions")) * 100,
    ),

    # Campaign status based on dates
    "campaign_status_derived": when(
        col("start_date").isNotNull() & col("end_date").isNotNull(),
        when(current_date() < col("start_date"), "Scheduled")
        .when(
            (current_date() >= col("start_date"))
            & (current_date() <= col("end_date")),
            "Active",
        )
        .otherwise("Completed"),
    ).otherwise(col("campaign_status")),

    # Days until end
    "days_until_end": when(
        col("end_date").isNotNull() & (col("end_date") >= current_date()),
        datediff(col("end_date"), current_date()),
    ),

    # Performance tier
    "performance_tier": when(
        col("roi").isNotNull(),
        when(col("roi") >= 200, "Excellent")
        .when(col("roi") >= 100, "Good")
        .when(col("roi") >= 0, "Break-even")
        .otherwise("Poor"),
    ),

    # Budget status
    "budget_status": when(
        col("budget_utilization_rate").isNotNull(),
        when(col("budget_utilization_rate") >= 100, "Over Budget")
        .when(col("budget_utilization_rate") >= 80, "Near Budget")
        .when(col("budget_utilization_rate") >= 50, "On Track")
        .otherwise("Under Budget"),
    ),
})

# 6. Campaign efficiency score (depends on ROI, CTR, conversion_rate)
dataframes["marketing_campaigns"] = dataframes["marketing_campaigns"].withColumn(
    "campaign_efficiency_score",
    when(
        col("roi").isNotNull()
        & col("ctr").isNotNull()
        & col("conversion_rate").isNotNull(),
        (
            (col("roi") / lit(100)) * lit(0.5)
            + (col("ctr") / lit(10)) * lit(0.25)
            + (col("conversion_rate") / lit(5)) * lit(0.25)
        )
        * lit(100),
    ),
).dropDuplicates(["campaign_id"])

## Time-Based

In [69]:
orders_with_time = (
    dataframes["orders"]
    .filter(col("order_placed_at").isNotNull())
    .withColumn("order_date", to_date(col("order_placed_at")))
    .withColumn("order_year", year(col("order_placed_at")))
    .withColumn("order_month", month(col("order_placed_at")))
    .withColumn("order_week", weekofyear(col("order_placed_at")))
    .withColumn("order_day_of_week", dayofweek(col("order_placed_at")))
    .withColumn(
        "year_month", expr("concat(order_year, '-', lpad(order_month, 2, '0'))")
    )
    .withColumn("year_week", expr("concat(order_year, '-W', lpad(order_week, 2, '0'))"))
)

In [70]:
orders_with_items_time = orders_with_time.join(
    dataframes["order_items"].select("order_id", "quantity"), "order_id", "left"
)

In [71]:
customer_first_order = (
    dataframes["orders"]
    .filter(col("order_placed_at").isNotNull())
    .groupBy("customer_id")
    .agg(spark_min("order_placed_at").alias("first_order_date"))
)

In [72]:
orders_with_customer_type = (
    orders_with_time.join(customer_first_order, "customer_id", "left")
    .withColumn(
        "is_new_customer",
        when(
            col("first_order_date").isNotNull()
            & (to_date(col("order_placed_at")) == to_date(col("first_order_date"))),
            lit(1),
        ).otherwise(lit(0)),
    )
    .withColumn(
        "is_returning_customer",
        when(col("is_new_customer") == 0, lit(1)).otherwise(lit(0)),
    )
)

In [73]:
daily_agg = orders_with_customer_type.groupBy(
    "order_date", "order_year", "order_month"
).agg(
    # Total revenue
    spark_sum(
        when(
            col("total_amount").isNotNull() & (col("total_amount") > 0),
            col("total_amount"),
        )
    ).alias("total_revenue"),
    # Total orders
    countDistinct("order_id").alias("total_orders"),
    # Total customers
    countDistinct("customer_id").alias("total_customers"),
    # New customers
    spark_sum("is_new_customer").alias("new_customers"),
    # Returning customers
    spark_sum("is_returning_customer").alias("returning_customers"),
    # Average order value
    spark_avg(
        when(
            col("total_amount").isNotNull() & (col("total_amount") > 0),
            col("total_amount"),
        )
    ).alias("avg_order_value"),
)

In [74]:
daily_units = orders_with_items_time.groupBy("order_date").agg(
    spark_sum(
        when(col("quantity").isNotNull() & (col("quantity") > 0), col("quantity"))
    ).alias("total_units_sold")
)

In [75]:
daily_agg = daily_agg.join(daily_units, "order_date", "left")

In [76]:
daily_sessions = (
    dataframes["customer_sessions"]
    .filter(col("session_start").isNotNull())
    .withColumn("session_date", to_date(col("session_start")))
    .groupBy("session_date")
    .agg(
        count("session_id").alias("total_sessions"),
        spark_sum(when(col("conversion_flag") == 1, lit(1)).otherwise(lit(0))).alias(
            "total_conversions"
        ),
    )
)

In [77]:
daily_agg = daily_agg.join(
    daily_sessions, daily_agg.order_date == daily_sessions.session_date, "left"
).drop("session_date")

In [78]:
window_daily = Window.orderBy("order_date")

daily_agg = daily_agg.withColumns(
    {
        # Session to order rate
        "session_to_order_rate": when(
            col("total_sessions").isNotNull() & (col("total_sessions") > 0),
            (col("total_orders") / col("total_sessions")) * 100,
        ),
        # Previous day metrics
        "prev_day_revenue": lag("total_revenue", 1).over(window_daily),
        "prev_day_customers": lag("total_customers", 1).over(window_daily),
        # Daily growth rate
        "revenue_growth_rate": when(
            col("prev_day_revenue").isNotNull() & (col("prev_day_revenue") > 0),
            ((col("total_revenue") - col("prev_day_revenue")) / col("prev_day_revenue"))
            * 100,
        ),
        # Customer retention rate
        "customer_retention_rate": when(
            col("prev_day_customers").isNotNull() & (col("prev_day_customers") > 0),
            (col("returning_customers") / col("prev_day_customers")) * 100,
        ),
        # Handle nulls
        "total_revenue": coalesce(col("total_revenue"), lit(0.0)),
        "total_orders": coalesce(col("total_orders"), lit(0)),
        "total_customers": coalesce(col("total_customers"), lit(0)),
        "new_customers": coalesce(col("new_customers"), lit(0)),
        "returning_customers": coalesce(col("returning_customers"), lit(0)),
        "total_units_sold": coalesce(col("total_units_sold"), lit(0)),
        "total_sessions": coalesce(col("total_sessions"), lit(0)),
    }
)

dataframes["daily_aggregations"] = daily_agg

In [79]:
weekly_agg = orders_with_customer_type.groupBy(
    "year_week", "order_year", "order_week"
).agg(
    spark_sum(
        when(
            col("total_amount").isNotNull() & (col("total_amount") > 0),
            col("total_amount"),
        )
    ).alias("total_revenue"),
    countDistinct("order_id").alias("total_orders"),
    countDistinct("customer_id").alias("total_customers"),
    spark_sum("is_new_customer").alias("new_customers"),
    spark_sum("is_returning_customer").alias("returning_customers"),
    spark_avg(
        when(
            col("total_amount").isNotNull() & (col("total_amount") > 0),
            col("total_amount"),
        )
    ).alias("avg_order_value"),
)

In [80]:
weekly_units = orders_with_items_time.groupBy("year_week").agg(
    spark_sum(
        when(col("quantity").isNotNull() & (col("quantity") > 0), col("quantity"))
    ).alias("total_units_sold")
)

In [81]:
weekly_agg = weekly_agg.join(weekly_units, "year_week", "left")

In [82]:
weekly_sessions = (
    dataframes["customer_sessions"]
    .filter(col("session_start").isNotNull())
    .withColumn("session_year", year(col("session_start")))
    .withColumn("session_week", weekofyear(col("session_start")))
    .withColumn(
        "session_year_week",
        expr("concat(session_year, '-W', lpad(session_week, 2, '0'))"),
    )
    .groupBy("session_year_week")
    .agg(
        count("session_id").alias("total_sessions"),
        spark_sum(when(col("conversion_flag") == 1, lit(1)).otherwise(lit(0))).alias(
            "total_conversions"
        ),
    )
)

In [83]:
weekly_agg = weekly_agg.join(
    weekly_sessions, weekly_agg.year_week == weekly_sessions.session_year_week, "left"
).drop("session_year_week")

In [84]:
window_weekly = Window.orderBy("order_year", "order_week")

In [85]:
weekly_agg = weekly_agg.withColumns(
    {
        "session_to_order_rate": when(
            col("total_sessions").isNotNull() & (col("total_sessions") > 0),
            (col("total_orders") / col("total_sessions")) * 100,
        ),
        "prev_week_revenue": lag("total_revenue", 1).over(window_weekly),
        "prev_week_customers": lag("total_customers", 1).over(window_weekly),
        "revenue_growth_rate": when(
            col("prev_week_revenue").isNotNull() & (col("prev_week_revenue") > 0),
            (
                (col("total_revenue") - col("prev_week_revenue"))
                / col("prev_week_revenue")
            )
            * 100,
        ),
        "customer_retention_rate": when(
            col("prev_week_customers").isNotNull() & (col("prev_week_customers") > 0),
            (col("returning_customers") / col("prev_week_customers")) * 100,
        ),
        "total_revenue": coalesce(col("total_revenue"), lit(0.0)),
        "total_orders": coalesce(col("total_orders"), lit(0)),
        "total_customers": coalesce(col("total_customers"), lit(0)),
        "new_customers": coalesce(col("new_customers"), lit(0)),
        "returning_customers": coalesce(col("returning_customers"), lit(0)),
        "total_units_sold": coalesce(col("total_units_sold"), lit(0)),
        "total_sessions": coalesce(col("total_sessions"), lit(0)),
    }
)

dataframes["weekly_aggregations"] = weekly_agg

In [86]:
monthly_agg = orders_with_customer_type.groupBy(
    "year_month", "order_year", "order_month"
).agg(
    spark_sum(
        when(
            col("total_amount").isNotNull() & (col("total_amount") > 0),
            col("total_amount"),
        )
    ).alias("total_revenue"),
    countDistinct("order_id").alias("total_orders"),
    countDistinct("customer_id").alias("total_customers"),
    spark_sum("is_new_customer").alias("new_customers"),
    spark_sum("is_returning_customer").alias("returning_customers"),
    spark_avg(
        when(
            col("total_amount").isNotNull() & (col("total_amount") > 0),
            col("total_amount"),
        )
    ).alias("avg_order_value"),
)

In [87]:
monthly_units = orders_with_items_time.groupBy("year_month").agg(
    spark_sum(
        when(col("quantity").isNotNull() & (col("quantity") > 0), col("quantity"))
    ).alias("total_units_sold")
)

In [88]:
monthly_agg = monthly_agg.join(monthly_units, "year_month", "left")

In [89]:
monthly_sessions = (
    dataframes["customer_sessions"]
    .filter(col("session_start").isNotNull())
    .withColumn("session_year", year(col("session_start")))
    .withColumn("session_month", month(col("session_start")))
    .withColumn(
        "session_year_month",
        expr("concat(session_year, '-', lpad(session_month, 2, '0'))"),
    )
    .groupBy("session_year_month")
    .agg(
        count("session_id").alias("total_sessions"),
        spark_sum(when(col("conversion_flag") == 1, lit(1)).otherwise(lit(0))).alias(
            "total_conversions"
        ),
    )
)

In [90]:
monthly_agg = monthly_agg.join(
    monthly_sessions,
    monthly_agg.year_month == monthly_sessions.session_year_month,
    "left",
).drop("session_year_month")

In [91]:
window_monthly = Window.orderBy("order_year", "order_month")

monthly_agg = monthly_agg.withColumns(
    {
        "session_to_order_rate": when(
            col("total_sessions").isNotNull() & (col("total_sessions") > 0),
            (col("total_orders") / col("total_sessions")) * 100,
        ),
        "prev_month_revenue": lag("total_revenue", 1).over(window_monthly),
        "prev_month_customers": lag("total_customers", 1).over(window_monthly),
        "revenue_growth_rate": when(
            col("prev_month_revenue").isNotNull() & (col("prev_month_revenue") > 0),
            (
                (col("total_revenue") - col("prev_month_revenue"))
                / col("prev_month_revenue")
            )
            * 100,
        ),
        "customer_retention_rate": when(
            col("prev_month_customers").isNotNull() & (col("prev_month_customers") > 0),
            (col("returning_customers") / col("prev_month_customers")) * 100,
        ),
        "churn_rate": when(
            col("prev_month_customers").isNotNull()
            & col("total_customers").isNotNull()
            & (col("prev_month_customers") > 0),
            (
                (col("prev_month_customers") - col("returning_customers"))
                / col("prev_month_customers")
            )
            * 100,
        ),
        "total_revenue": coalesce(col("total_revenue"), lit(0.0)),
        "total_orders": coalesce(col("total_orders"), lit(0)),
        "total_customers": coalesce(col("total_customers"), lit(0)),
        "new_customers": coalesce(col("new_customers"), lit(0)),
        "returning_customers": coalesce(col("returning_customers"), lit(0)),
        "total_units_sold": coalesce(col("total_units_sold"), lit(0)),
        "total_sessions": coalesce(col("total_sessions"), lit(0)),
    }
)

dataframes["monthly_aggregations"] = monthly_agg

## Geographic

In [92]:
orders_with_geography = dataframes["orders"].join(
    dataframes["customers"].select("customer_id", "city", "state_province", "country"),
    "customer_id",
    "left",
)

In [93]:
orders_with_category = orders_with_geography.join(
    dataframes["order_items"].select("order_id", "product_id"), "order_id", "left"
).join(dataframes["products"].select("product_id", "category"), "product_id", "left")

In [94]:
country_agg = (
    orders_with_geography.filter(col("country").isNotNull())
    .groupBy("country")
    .agg(
        # Total customers
        countDistinct("customer_id").alias("total_customers"),
        # Total orders
        countDistinct("order_id").alias("total_orders"),
        # Total revenue
        spark_sum(
            when(
                col("total_amount").isNotNull() & (col("total_amount") > 0),
                col("total_amount"),
            )
        ).alias("total_revenue"),
        # Average order value
        spark_avg(
            when(
                col("total_amount").isNotNull() & (col("total_amount") > 0),
                col("total_amount"),
            )
        ).alias("avg_order_value"),
    )
)

In [95]:
country_clv = (
    dataframes["customers"]
    .filter(col("country").isNotNull())
    .groupBy("country")
    .agg(
        spark_avg(
            when(
                col("customer_lifetime_value").isNotNull(),
                col("customer_lifetime_value"),
            )
        ).alias("avg_customer_lifetime_value")
    )
)

country_agg = country_agg.join(country_clv, "country", "left")

In [96]:
country_suppliers = (
    dataframes["suppliers"]
    .filter(col("country").isNotNull())
    .groupBy("country")
    .agg(countDistinct("supplier_id").alias("total_suppliers"))
)

country_agg = country_agg.join(country_suppliers, "country", "left")

In [97]:
country_categories = (
    orders_with_category.filter(
        col("country").isNotNull() & col("category").isNotNull()
    )
    .groupBy("country", "category")
    .agg(
        spark_sum(
            when(
                col("total_amount").isNotNull() & (col("total_amount") > 0),
                col("total_amount"),
            )
        ).alias("category_revenue")
    )
)

In [98]:
window_country_category = Window.partitionBy("country").orderBy(
    col("category_revenue").desc()
)

country_top_category = (
    country_categories.withColumn("rank", row_number().over(window_country_category))
    .filter(col("rank") == 1)
    .select("country", col("category").alias("preferred_category"))
)

country_agg = country_agg.join(country_top_category, "country", "left")

In [99]:
country_agg = country_agg.withColumns(
    {
        "revenue_per_customer": when(
            col("total_revenue").isNotNull()
            & col("total_customers").isNotNull()
            & (col("total_customers") > 0),
            col("total_revenue") / col("total_customers"),
        ),
        "orders_per_customer": when(
            col("total_orders").isNotNull()
            & col("total_customers").isNotNull()
            & (col("total_customers") > 0),
            col("total_orders") / col("total_customers"),
        ),
        "total_customers": coalesce(col("total_customers"), lit(0)),
        "total_orders": coalesce(col("total_orders"), lit(0)),
        "total_revenue": coalesce(col("total_revenue"), lit(0.0)),
        "total_suppliers": coalesce(col("total_suppliers"), lit(0)),
    }
)

dataframes["country_aggregations"] = country_agg

In [100]:
state_agg = (
    orders_with_geography.filter(
        col("state_province").isNotNull() & col("country").isNotNull()
    )
    .groupBy("country", "state_province")
    .agg(
        countDistinct("customer_id").alias("total_customers"),
        countDistinct("order_id").alias("total_orders"),
        spark_sum(
            when(
                col("total_amount").isNotNull() & (col("total_amount") > 0),
                col("total_amount"),
            )
        ).alias("total_revenue"),
        spark_avg(
            when(
                col("total_amount").isNotNull() & (col("total_amount") > 0),
                col("total_amount"),
            )
        ).alias("avg_order_value"),
    )
)

In [101]:
state_clv = (
    dataframes["customers"]
    .filter(col("state_province").isNotNull() & col("country").isNotNull())
    .groupBy("country", "state_province")
    .agg(
        spark_avg(
            when(
                col("customer_lifetime_value").isNotNull(),
                col("customer_lifetime_value"),
            )
        ).alias("avg_customer_lifetime_value")
    )
)
state_agg = state_agg.join(state_clv, ["country", "state_province"], "left")

In [102]:
state_suppliers = (
    dataframes["suppliers"]
    .filter(col("state").isNotNull() & col("country").isNotNull())
    .groupBy(col("country"), col("state").alias("state_province"))
    .agg(countDistinct("supplier_id").alias("total_suppliers"))
)

state_agg = state_agg.join(state_suppliers, ["country", "state_province"], "left")

In [103]:
state_categories = (
    orders_with_category.filter(
        col("state_province").isNotNull()
        & col("country").isNotNull()
        & col("category").isNotNull()
    )
    .groupBy("country", "state_province", "category")
    .agg(
        spark_sum(
            when(
                col("total_amount").isNotNull() & (col("total_amount") > 0),
                col("total_amount"),
            )
        ).alias("category_revenue")
    )
)

In [104]:
window_state_category = Window.partitionBy("country", "state_province").orderBy(
    col("category_revenue").desc()
)

state_top_category = (
    state_categories.withColumn("rank", row_number().over(window_state_category))
    .filter(col("rank") == 1)
    .select("country", "state_province", col("category").alias("preferred_category"))
)

state_agg = state_agg.join(state_top_category, ["country", "state_province"], "left")

In [105]:
state_agg = state_agg.withColumns(
    {
        "revenue_per_customer": when(
            col("total_revenue").isNotNull()
            & col("total_customers").isNotNull()
            & (col("total_customers") > 0),
            col("total_revenue") / col("total_customers"),
        ),
        "orders_per_customer": when(
            col("total_orders").isNotNull()
            & col("total_customers").isNotNull()
            & (col("total_customers") > 0),
            col("total_orders") / col("total_customers"),
        ),
        "total_customers": coalesce(col("total_customers"), lit(0)),
        "total_orders": coalesce(col("total_orders"), lit(0)),
        "total_revenue": coalesce(col("total_revenue"), lit(0.0)),
        "total_suppliers": coalesce(col("total_suppliers"), lit(0)),
    }
)

dataframes["state_aggregations"] = state_agg

In [106]:
city_agg = (
    orders_with_geography.filter(
        col("city").isNotNull()
        & col("state_province").isNotNull()
        & col("country").isNotNull()
    )
    .groupBy("country", "state_province", "city")
    .agg(
        countDistinct("customer_id").alias("total_customers"),
        countDistinct("order_id").alias("total_orders"),
        spark_sum(
            when(
                col("total_amount").isNotNull() & (col("total_amount") > 0),
                col("total_amount"),
            )
        ).alias("total_revenue"),
        spark_avg(
            when(
                col("total_amount").isNotNull() & (col("total_amount") > 0),
                col("total_amount"),
            )
        ).alias("avg_order_value"),
    )
)

In [107]:
city_clv = (
    dataframes["customers"]
    .filter(
        col("city").isNotNull()
        & col("state_province").isNotNull()
        & col("country").isNotNull()
    )
    .groupBy("country", "state_province", "city")
    .agg(
        spark_avg(
            when(
                col("customer_lifetime_value").isNotNull(),
                col("customer_lifetime_value"),
            )
        ).alias("avg_customer_lifetime_value")
    )
)

city_agg = city_agg.join(city_clv, ["country", "state_province", "city"], "left")

In [108]:
city_suppliers = (
    dataframes["suppliers"]
    .filter(
        col("city").isNotNull() & col("state").isNotNull() & col("country").isNotNull()
    )
    .groupBy(col("country"), col("state").alias("state_province"), col("city"))
    .agg(countDistinct("supplier_id").alias("total_suppliers"))
)

city_agg = city_agg.join(city_suppliers, ["country", "state_province", "city"], "left")

In [109]:
city_categories = (
    orders_with_category.filter(
        col("city").isNotNull()
        & col("state_province").isNotNull()
        & col("country").isNotNull()
        & col("category").isNotNull()
    )
    .groupBy("country", "state_province", "city", "category")
    .agg(
        spark_sum(
            when(
                col("total_amount").isNotNull() & (col("total_amount") > 0),
                col("total_amount"),
            )
        ).alias("category_revenue")
    )
)

In [110]:
from pyspark.sql.functions import broadcast

# Persist city_categories to avoid recomputation
city_categories.persist()

window_city_category = Window.partitionBy("country", "state_province", "city").orderBy(
    col("category_revenue").desc()
)

# Use broadcast hint for smaller dataframe and repartition for better parallelism
city_top_category = (
    city_categories.withColumn("rank", row_number().over(window_city_category))
    .filter(col("rank") == 1)
    .select(
        "country", "state_province", "city", col("category").alias("preferred_category")
    )
    .repartition(200)  # Adjust based on your cluster size
)

# Perform the join with broadcast hint if city_top_category is small
city_agg = city_agg.join(
    broadcast(city_top_category), ["country", "state_province", "city"], "left"
)

# Unpersist after use to free memory
city_categories.unpersist()

DataFrame[country: string, state_province: string, city: string, category: string, category_revenue: double]

In [111]:
city_agg = city_agg.withColumns(
    {
        "revenue_per_customer": when(
            col("total_revenue").isNotNull()
            & col("total_customers").isNotNull()
            & (col("total_customers") > 0),
            col("total_revenue") / col("total_customers"),
        ),
        "orders_per_customer": when(
            col("total_orders").isNotNull()
            & col("total_customers").isNotNull()
            & (col("total_customers") > 0),
            col("total_orders") / col("total_customers"),
        ),
        "customer_density": col("total_customers"),  # Density = customers in that city
        "total_customers": coalesce(col("total_customers"), lit(0)),
        "total_orders": coalesce(col("total_orders"), lit(0)),
        "total_revenue": coalesce(col("total_revenue"), lit(0.0)),
        "total_suppliers": coalesce(col("total_suppliers"), lit(0)),
    }
)

dataframes["city_aggregations"] = city_agg

## Session

In [112]:
session_cart_agg = (
    dataframes["shopping_cart"]
    .filter(col("session_id").isNotNull())
    .groupBy("session_id")
    .agg(
        # Items added to cart
        count("cart_id").alias("items_added_to_cart"),
        # Cart value (unit_price * quantity)
        spark_sum(
            when(
                col("unit_price").isNotNull()
                & col("quantity").isNotNull()
                & (col("unit_price") > 0)
                & (col("quantity") > 0),
                col("unit_price") * col("quantity"),
            )
        ).alias("cart_value"),
    )
)

In [113]:
session_orders = (
    dataframes["customer_sessions"]
    .join(
        dataframes["orders"].select("customer_id", "order_id", "order_placed_at"),
        "customer_id",
        "left",
    )
    .filter(
        col("order_placed_at").isNotNull()
        & (col("order_placed_at") >= col("session_start"))
        & (col("order_placed_at") <= col("session_end"))
    )
    .groupBy("session_id")
    .agg(countDistinct("order_id").alias("orders_from_session"))
)

In [114]:
dataframes["customer_sessions"] = (
    dataframes["customer_sessions"]
    .join(session_cart_agg, "session_id", "left")
    .join(session_orders, "session_id", "left")
)

In [115]:
dataframes["customer_sessions"] = dataframes["customer_sessions"].withColumns(
    {
        # Session duration (already calculated, ensure exists)
        "session_duration_minutes": coalesce(col("session_duration_minutes"), lit(0)),
        # Total pages viewed (already exists)
        "total_pages_viewed": coalesce(col("pages_viewed"), lit(0)),
        # Total products viewed (already exists)
        "total_products_viewed": coalesce(col("products_viewed"), lit(0)),
        # Items added to cart (from join)
        "items_added_to_cart": coalesce(col("items_added_to_cart"), lit(0)),
        # Cart value (from join)
        "cart_value": coalesce(col("cart_value"), lit(0.0)),
        # Converted flag (1 if order placed, 0 otherwise)
        "converted": when(
            col("orders_from_session").isNotNull() & (col("orders_from_session") > 0),
            lit(1),
        ).otherwise(lit(0)),
        # Abandoned flag (already exists as cart_abandonment_flag)
        "abandoned": coalesce(col("cart_abandonment_flag"), lit(0)),
        # Device type (already exists)
        "device_type": col("device_type"),
        # Referrer source (already exists)
        "referrer_source": col("referrer_source"),
        # Pages per minute (already calculated, ensure exists)
        "pages_per_minute": when(
            col("session_duration_minutes").isNotNull()
            & (col("session_duration_minutes") > 0)
            & col("pages_viewed").isNotNull(),
            col("pages_viewed") / col("session_duration_minutes"),
        ).otherwise(lit(0.0)),
        # Additional useful metrics
        "products_per_page": when(
            col("pages_viewed").isNotNull()
            & (col("pages_viewed") > 0)
            & col("products_viewed").isNotNull(),
            col("products_viewed") / col("pages_viewed"),
        ).otherwise(lit(0.0)),
        "cart_add_rate": when(
            col("products_viewed").isNotNull()
            & (col("products_viewed") > 0)
            & col("items_added_to_cart").isNotNull(),
            (col("items_added_to_cart") / col("products_viewed")) * 100,
        ).otherwise(lit(0.0)),
        "avg_cart_item_value": when(
            col("cart_value").isNotNull()
            & col("items_added_to_cart").isNotNull()
            & (col("items_added_to_cart") > 0),
            col("cart_value") / col("items_added_to_cart"),
        ).otherwise(lit(0.0)),
        "session_engagement_score": when(
            col("session_duration_minutes").isNotNull()
            & col("products_viewed").isNotNull()
            & col("items_added_to_cart").isNotNull(),
            (
                # Duration weight (30%)
                (col("session_duration_minutes") / lit(60)) * lit(0.3)
                +
                # Product views weight (40%)
                (col("products_viewed") / lit(10)) * lit(0.4)
                +
                # Cart additions weight (30%)
                (col("items_added_to_cart") / lit(5)) * lit(0.3)
            )
            * lit(100),
        ).otherwise(lit(0.0)),
        "session_type": when(col("converted") == 1, "Converted")
        .when(col("abandoned") == 1, "Abandoned")
        .when(col("items_added_to_cart") > 0, "Cart Activity")
        .when(col("products_viewed") > 0, "Browsing")
        .otherwise("Bounce"),
    }
).dropDuplicates(["session_id"])

## Cart Abandonment

In [116]:
cart_with_products = (
    dataframes["shopping_cart"]
    .join(dataframes["products"].select("product_id", "category"), "product_id", "left")
    .join(
        dataframes["customer_sessions"].select(
            "session_id", "device_type", "conversion_flag"
        ),
        "session_id",
        "left",
    )
)

In [117]:
cart_agg = (
    cart_with_products.filter(col("cart_id").isNotNull())
    .groupBy("cart_id")
    .agg(
        # Cart total value
        spark_sum(
            when(
                col("unit_price").isNotNull()
                & col("quantity").isNotNull()
                & (col("unit_price") > 0)
                & (col("quantity") > 0),
                col("unit_price") * col("quantity"),
            )
        ).alias("cart_total_value"),
        # Cart items count
        count("product_id").alias("cart_items_count"),
        # Average item price
        spark_avg(
            when(
                col("unit_price").isNotNull() & (col("unit_price") > 0),
                col("unit_price"),
            )
        ).alias("cart_avg_item_price"),
        # Device used
        expr("first(device_type)").alias("device_used"),
        # Primary category (most frequent)
        expr("first(category)").alias("abandoned_cart_category"),
        # Session conversion flag
        expr("first(conversion_flag)").alias("session_converted"),
        # Earliest added date
        spark_min("added_date").alias("first_added_date"),
        # Latest added date
        spark_max("added_date").alias("last_added_date"),
        # Customer ID
        expr("first(customer_id)").alias("customer_id"),
        # Session ID
        expr("first(session_id)").alias("session_id"),
    )
)

In [118]:
cart_full = (
    dataframes["shopping_cart"]
    .select("cart_id", "cart_status", "added_date")
    .groupBy("cart_id")
    .agg(
        expr("first(cart_status)").alias("cart_status"),
        spark_min("added_date").alias("cart_added_date"),
    )
    .join(cart_agg, "cart_id", "left")
)

In [119]:
cart_full = cart_full.withColumns(
    {
        # Time in cart (days since first added)
        "time_in_cart_days": when(
            col("first_added_date").isNotNull(),
            datediff(current_date(), col("first_added_date")),
        ),
        # Time in cart (hours)
        "time_in_cart_hours": when(
            col("first_added_date").isNotNull(),
            datediff(current_date(), col("first_added_date")) * 24,
        ),
        # Cart status classification
        "cart_status_derived": when(
            lower(col("cart_status")).isin("purchased", "completed", "ordered"),
            "Purchased",
        )
        .when(lower(col("cart_status")).isin("active", "in progress"), "Active")
        .otherwise("Abandoned"),
        # Handle nulls for aggregated columns
        "cart_total_value": coalesce(col("cart_total_value"), lit(0.0)),
        "cart_items_count": coalesce(col("cart_items_count"), lit(0)),
        "cart_avg_item_price": coalesce(col("cart_avg_item_price"), lit(0.0)),
    }
)

In [120]:
cart_full = cart_full.withColumns(
    {
        # Cart abandonment reason (derived from patterns)
        "cart_abandonment_reason": when(
            col("cart_status_derived") == "Purchased", "N/A - Purchased"
        )
        .when(col("cart_status_derived") == "Active", "N/A - Active")
        .when(col("cart_total_value") > 500, "High Value - Price Concern")
        .when(col("cart_items_count") > 10, "Too Many Items")
        .when(col("time_in_cart_days") > 7, "Long Time - Lost Interest")
        .when(col("time_in_cart_days") <= 1, "Quick Abandon - Browsing")
        .when(col("device_used") == "Mobile", "Mobile - Checkout Friction")
        .otherwise("Unknown"),
        # Cart value tier
        "cart_value_tier": when(col("cart_total_value") >= 1000, "High Value")
        .when(col("cart_total_value") >= 500, "Medium Value")
        .when(col("cart_total_value") >= 100, "Low Value")
        .otherwise("Very Low Value"),
        # Cart size category
        "cart_size_category": when(col("cart_items_count") >= 10, "Large")
        .when(col("cart_items_count") >= 5, "Medium")
        .when(col("cart_items_count") >= 2, "Small")
        .otherwise("Single Item"),
        # Abandonment risk score
        "abandonment_risk_score": when(
            col("cart_status_derived") != "Purchased",
            (
                # Time weight (40%)
                (col("time_in_cart_days") / lit(30)) * lit(0.4)
                +
                # Value weight (30%)
                (col("cart_total_value") / lit(1000)) * lit(0.3)
                +
                # Items weight (30%)
                (col("cart_items_count") / lit(10)) * lit(0.3)
            )
            * lit(100),
        ).otherwise(lit(0.0)),
        # Recovery potential score
        "recovery_potential_score": when(
            col("cart_status_derived") == "Abandoned",
            when(
                (col("cart_total_value") > 200) & (col("time_in_cart_days") <= 3),
                lit(100),
            )
            .when(
                (col("cart_total_value") > 100) & (col("time_in_cart_days") <= 7),
                lit(75),
            )
            .when(col("time_in_cart_days") <= 14, lit(50))
            .otherwise(lit(25)),
        ).otherwise(lit(0)),
    }
)

In [121]:
dataframes["cart_abandonment_analysis"] = cart_full

In [122]:
customer_cart_behavior = (
    cart_full.filter(col("customer_id").isNotNull())
    .groupBy("customer_id")
    .agg(
        # Total carts created
        count("cart_id").alias("total_carts_created"),
        # Abandoned carts
        spark_sum(
            when(col("cart_status_derived") == "Abandoned", lit(1)).otherwise(lit(0))
        ).alias("total_abandoned_carts"),
        # Purchased carts
        spark_sum(
            when(col("cart_status_derived") == "Purchased", lit(1)).otherwise(lit(0))
        ).alias("total_purchased_carts"),
        # Total abandoned value
        spark_sum(
            when(col("cart_status_derived") == "Abandoned", col("cart_total_value"))
        ).alias("total_abandoned_value"),
        # Average time in cart
        spark_avg("time_in_cart_days").alias("avg_time_in_cart_days"),
    )
    .withColumns(
        {
            # Customer abandonment rate
            "customer_abandonment_rate": when(
                col("total_carts_created") > 0,
                (col("total_abandoned_carts") / col("total_carts_created")) * 100,
            ),
            # Customer purchase rate
            "customer_purchase_rate": when(
                col("total_carts_created") > 0,
                (col("total_purchased_carts") / col("total_carts_created")) * 100,
            ),
        }
    )
)

In [123]:
dataframes["customers"] = (
    dataframes["customers"]
    .drop(
        "total_carts_created",
        "total_abandoned_carts",
        "total_abandoned_value",
        "avg_time_in_cart_days",
        "customer_abandonment_rate",
        "customer_purchase_rate",
    )
    .join(customer_cart_behavior, "customer_id", "left")
    .withColumns(
        {
            "total_carts_created": coalesce(col("total_carts_created"), lit(0)),
            "total_abandoned_carts": coalesce(col("total_abandoned_carts"), lit(0)),
            "total_abandoned_value": coalesce(col("total_abandoned_value"), lit(0.0)),
        }
    )
)

## Inventory Health

In [124]:
product_daily_sales = (
    dataframes["orders"]
    .filter(col("order_placed_at").isNotNull())
    .join(dataframes["order_items"], "order_id", "inner")
    .withColumn("order_date", to_date(col("order_placed_at")))
    .groupBy("product_id", "order_date")
    .agg(
        spark_sum(
            when(col("quantity").isNotNull() & (col("quantity") > 0), col("quantity"))
        ).alias("daily_quantity")
    )
    .groupBy("product_id")
    .agg(spark_avg("daily_quantity").alias("avg_daily_sales"))
)

In [125]:
inventory_enhanced = (
    dataframes["inventory"]
    .join(
        dataframes["products"].select(
            col("product_id").alias("prod_product_id"),
            col("supplier_id").alias("prod_supplier_id"),
            col("cost_price").alias("prod_cost_price"),
        ),
        dataframes["inventory"]["product_id"] == col("prod_product_id"),
        "left",
    )
    .drop("prod_product_id")
    .withColumn("supplier_id", coalesce(col("prod_supplier_id"), col("supplier_id")))
    .withColumn("cost_price", coalesce(col("prod_cost_price"), lit(0.0)))
    .drop("prod_supplier_id", "prod_cost_price")
    .join(product_daily_sales, "product_id", "left")
)

In [126]:
product_inventory_health = (
    inventory_enhanced.filter(col("product_id").isNotNull())
    .groupBy("product_id")
    .agg(
        # Current stock (latest value)
        expr("last(stock_quantity)").alias("current_stock"),
        # Available stock (latest)
        expr("last(stock_quantity - reserved_quantity)").alias("available_stock"),
        # Average stock quantity
        spark_avg(
            when(
                col("stock_quantity").isNotNull() & (col("stock_quantity") >= 0),
                col("stock_quantity"),
            )
        ).alias("avg_stock_quantity"),
        # Reorder point breach count
        spark_sum(
            when(
                col("stock_quantity").isNotNull()
                & col("minimum_stock_level").isNotNull()
                & (col("stock_quantity") < col("minimum_stock_level")),
                lit(1),
            ).otherwise(lit(0))
        ).alias("reorder_point_breach_count"),
        # Stockout frequency
        spark_sum(
            when(
                col("stock_quantity").isNotNull() & (col("stock_quantity") == 0), lit(1)
            ).otherwise(lit(0))
        ).alias("stockout_frequency"),
        # Storage cost per unit (latest)
        expr("last(storage_cost / NULLIF(stock_quantity, 0))").alias(
            "storage_cost_per_unit"
        ),
        # Last restock date
        spark_max("last_restocked_date").alias("last_restock_date"),
        # Latest reserved quantity
        expr("last(reserved_quantity)").alias("reserved_quantity"),
        # Latest minimum stock level
        expr("last(minimum_stock_level)").alias("minimum_stock_level"),
        # Latest storage cost
        expr("last(storage_cost)").alias("storage_cost"),
        # Supplier ID (latest)
        expr("last(supplier_id)").alias("supplier_id"),
        # Cost price (latest)
        expr("last(cost_price)").alias("cost_price"),
        # Average daily sales
        expr("last(avg_daily_sales)").alias("avg_daily_sales"),
    )
)

In [127]:
product_cogs = (
    dataframes["order_items"]
    .filter(col("product_id").isNotNull())
    .groupBy("product_id")
    .agg(
        spark_sum(
            when(
                col("quantity").isNotNull()
                & col("product_cost").isNotNull()
                & (col("quantity") > 0)
                & (col("product_cost") > 0),
                col("quantity") * col("product_cost"),
            )
        ).alias("total_cogs")
    )
)

In [128]:
product_inventory_health = product_inventory_health.join(
    product_cogs, "product_id", "left"
)

In [129]:
product_inventory_health = product_inventory_health.withColumns(
    {
        # Stock status
        "stock_status": when(col("current_stock") == 0, "Out of Stock")
        .when(col("current_stock") < col("minimum_stock_level"), "Low Stock")
        .when(col("current_stock") > (col("minimum_stock_level") * 3), "Overstock")
        .otherwise("Adequate"),
        # Days of supply
        "days_of_supply": when(
            col("current_stock").isNotNull()
            & col("avg_daily_sales").isNotNull()
            & (col("avg_daily_sales") > 0),
            col("current_stock") / col("avg_daily_sales"),
        ),
        # Inventory turnover ratio
        "inventory_turnover_ratio": when(
            col("total_cogs").isNotNull()
            & col("avg_stock_quantity").isNotNull()
            & (col("avg_stock_quantity") > 0),
            col("total_cogs") / col("avg_stock_quantity"),
        ),
        # Days since restock
        "days_since_restock": when(
            col("last_restock_date").isNotNull(),
            datediff(current_date(), col("last_restock_date")),
        ),
        # Stock health score
        "stock_health_score": when(
            col("current_stock").isNotNull() & col("minimum_stock_level").isNotNull(),
            when(col("stock_status") == "Adequate", lit(100))
            .when(col("stock_status") == "Overstock", lit(75))
            .when(col("stock_status") == "Low Stock", lit(50))
            .when(col("stock_status") == "Out of Stock", lit(0)),
        ),
        # Reorder urgency
        "reorder_urgency": when(col("stock_status") == "Out of Stock", "Critical")
        .when(
            (col("stock_status") == "Low Stock") & (col("days_of_supply") < 7), "High"
        )
        .when(
            (col("stock_status") == "Low Stock") & (col("days_of_supply") < 14),
            "Medium",
        )
        .otherwise("Low"),
        # Handle nulls
        "current_stock": coalesce(col("current_stock"), lit(0)),
        "available_stock": coalesce(col("available_stock"), lit(0)),
        "reorder_point_breach_count": coalesce(
            col("reorder_point_breach_count"), lit(0)
        ),
        "stockout_frequency": coalesce(col("stockout_frequency"), lit(0)),
        "storage_cost_per_unit": coalesce(col("storage_cost_per_unit"), lit(0.0)),
    }
)

dataframes["product_inventory_health"] = product_inventory_health

In [130]:
supplier_inventory_health = (
    inventory_enhanced.filter(col("supplier_id").isNotNull())
    .groupBy("supplier_id")
    .agg(
        # Total products
        countDistinct("product_id").alias("total_products"),
        # Total current stock
        spark_sum(
            when(
                col("stock_quantity").isNotNull() & (col("stock_quantity") >= 0),
                col("stock_quantity"),
            )
        ).alias("total_current_stock"),
        # Total available stock
        spark_sum(
            when(
                col("stock_quantity").isNotNull()
                & col("reserved_quantity").isNotNull(),
                col("stock_quantity") - col("reserved_quantity"),
            )
        ).alias("total_available_stock"),
        # Reorder point breaches
        spark_sum(
            when(
                col("stock_quantity").isNotNull()
                & col("minimum_stock_level").isNotNull()
                & (col("stock_quantity") < col("minimum_stock_level")),
                lit(1),
            ).otherwise(lit(0))
        ).alias("total_reorder_breaches"),
        # Stockout count
        spark_sum(
            when(
                col("stock_quantity").isNotNull() & (col("stock_quantity") == 0), lit(1)
            ).otherwise(lit(0))
        ).alias("total_stockouts"),
        # Total storage cost
        spark_sum(
            when(
                col("storage_cost").isNotNull() & (col("storage_cost") > 0),
                col("storage_cost"),
            )
        ).alias("total_storage_cost"),
        # Average stock quantity
        spark_avg(
            when(
                col("stock_quantity").isNotNull() & (col("stock_quantity") >= 0),
                col("stock_quantity"),
            )
        ).alias("avg_stock_per_product"),
        # Latest restock date
        spark_max("last_restocked_date").alias("last_restock_date"),
    )
)

In [131]:
supplier_inventory_health = supplier_inventory_health.withColumns(
    {
        # Stockout rate
        "stockout_rate": when(
            col("total_products") > 0,
            (col("total_stockouts") / col("total_products")) * 100,
        ),
        # Breach rate
        "breach_rate": when(
            col("total_products") > 0,
            (col("total_reorder_breaches") / col("total_products")) * 100,
        ),
        # Average storage cost per unit
        "avg_storage_cost_per_unit": when(
            col("total_current_stock") > 0,
            col("total_storage_cost") / col("total_current_stock"),
        ),
        # Days since last restock
        "days_since_last_restock": when(
            col("last_restock_date").isNotNull(),
            datediff(current_date(), col("last_restock_date")),
        ),
        # Supplier inventory health score
        "supplier_inventory_health_score": when(
            col("stockout_rate").isNotNull() & col("breach_rate").isNotNull(),
            (
                # Low stockout rate (50%)
                (lit(100) - col("stockout_rate")) * lit(0.5)
                +
                # Low breach rate (50%)
                (lit(100) - col("breach_rate")) * lit(0.5)
            ),
        ),
        # Handle nulls
        "total_products": coalesce(col("total_products"), lit(0)),
        "total_current_stock": coalesce(col("total_current_stock"), lit(0)),
        "total_available_stock": coalesce(col("total_available_stock"), lit(0)),
        "total_reorder_breaches": coalesce(col("total_reorder_breaches"), lit(0)),
        "total_stockouts": coalesce(col("total_stockouts"), lit(0)),
        "total_storage_cost": coalesce(col("total_storage_cost"), lit(0.0)),
    }
)

dataframes["supplier_inventory_health"] = supplier_inventory_health

In [132]:
dataframes["products"] = (
    dataframes["products"]
    .join(
        product_inventory_health.select(
            col("product_id").alias("inv_product_id"),
            col("current_stock").alias("inv_current_stock"),
            col("stock_status").alias("inv_stock_status"),
            col("days_of_supply").alias("inv_days_of_supply"),
            col("reorder_urgency").alias("inv_reorder_urgency"),
        ),
        dataframes["products"]["product_id"] == col("inv_product_id"),
        "left",
    )
    .drop(
        "inv_product_id",
        "current_stock",
        "stock_status",
        "days_of_supply",
        "reorder_urgency",
    )
    .withColumnRenamed("inv_current_stock", "current_stock")
    .withColumnRenamed("inv_stock_status", "stock_status")
    .withColumnRenamed("inv_days_of_supply", "days_of_supply")
    .withColumnRenamed("inv_reorder_urgency", "reorder_urgency")
).dropDuplicates(["product_id"])

In [133]:
dataframes["suppliers"] = (
    dataframes["suppliers"]
    .join(
        supplier_inventory_health.select(
            col("supplier_id").alias("inv_supplier_id"),
            col("total_stockouts").alias("inv_total_stockouts"),
            col("stockout_rate").alias("inv_stockout_rate"),
            col("supplier_inventory_health_score").alias("inv_supplier_health_score"),
        ),
        dataframes["suppliers"]["supplier_id"] == col("inv_supplier_id"),
        "left",
    )
    .drop(
        "inv_supplier_id",
        "total_stockouts",
        "stockout_rate",
        "supplier_inventory_health_score",
    )
    .withColumnRenamed("inv_total_stockouts", "total_stockouts")
    .withColumnRenamed("inv_stockout_rate", "stockout_rate")
    .withColumnRenamed("inv_supplier_health_score", "supplier_inventory_health_score")
).dropDuplicates(["supplier_id"])

## RFM Segmentation

In [134]:
rfm_metrics = (
    dataframes["orders"]
    .filter(col("customer_id").isNotNull() & col("order_placed_at").isNotNull())
    .groupBy("customer_id")
    .agg(
        # Recency: Days since last order
        datediff(lit("2025-11-14"), spark_max("order_placed_at")).alias(
            "days_since_last_order"
        ),
        # Frequency: Total number of orders
        countDistinct("order_id").alias("total_orders_rfm"),
        # Monetary: Total revenue
        spark_sum(
            when(
                col("total_amount").isNotNull() & (col("total_amount") > 0),
                col("total_amount"),
            )
        ).alias("total_revenue_rfm"),
    )
)

In [135]:
recency_percentiles = rfm_metrics.approxQuantile(
    "days_since_last_order", [0.2, 0.4, 0.6, 0.8], 0.01
)
frequency_percentiles = rfm_metrics.approxQuantile(
    "total_orders_rfm", [0.2, 0.4, 0.6, 0.8], 0.01
)
monetary_percentiles = rfm_metrics.approxQuantile(
    "total_revenue_rfm", [0.2, 0.4, 0.6, 0.8], 0.01
)

In [136]:
# Check if percentiles are valid before using them
if len(recency_percentiles) == 4 and len(frequency_percentiles) == 4 and len(monetary_percentiles) == 4:
    rfm_scored = rfm_metrics.withColumns(
        {
            # Recency Score (1-5, lower days = higher score)
            "recency_score": when(
                col("days_since_last_order") <= recency_percentiles[0], lit(5)
            )
            .when(col("days_since_last_order") <= recency_percentiles[1], lit(4))
            .when(col("days_since_last_order") <= recency_percentiles[2], lit(3))
            .when(col("days_since_last_order") <= recency_percentiles[3], lit(2))
            .otherwise(lit(1)),
            # Frequency Score (1-5, higher orders = higher score)
            "frequency_score": when(
                col("total_orders_rfm") >= frequency_percentiles[3], lit(5)
            )
            .when(col("total_orders_rfm") >= frequency_percentiles[2], lit(4))
            .when(col("total_orders_rfm") >= frequency_percentiles[1], lit(3))
            .when(col("total_orders_rfm") >= frequency_percentiles[0], lit(2))
            .otherwise(lit(1)),
            # Monetary Score (1-5, higher revenue = higher score)
            "monetary_score": when(
                col("total_revenue_rfm") >= monetary_percentiles[3], lit(5)
            )
            .when(col("total_revenue_rfm") >= monetary_percentiles[2], lit(4))
            .when(col("total_revenue_rfm") >= monetary_percentiles[1], lit(3))
            .when(col("total_revenue_rfm") >= monetary_percentiles[0], lit(2))
            .otherwise(lit(1)),
        }
    )
else:
    # If percentiles calculation failed, assign default scores
    rfm_scored = rfm_metrics.withColumns(
        {
            "recency_score": lit(3),
            "frequency_score": lit(3),
            "monetary_score": lit(3),
        }
    )

In [137]:
rfm_scored = rfm_scored.withColumn(
    "rfm_segment", expr("concat(recency_score, frequency_score, monetary_score)")
)

In [138]:
rfm_scored = rfm_scored.withColumn(
    "customer_segment_label",
    when(
        (col("recency_score") >= 4)
        & (col("frequency_score") >= 4)
        & (col("monetary_score") >= 4),
        "Champions",
    )
    .when(
        (col("recency_score") >= 3)
        & (col("frequency_score") >= 3)
        & (col("monetary_score") >= 3),
        "Loyal Customers",
    )
    .when(
        (col("recency_score") >= 4)
        & (col("frequency_score") <= 2)
        & (col("monetary_score") <= 2),
        "Promising",
    )
    .when(
        (col("recency_score") >= 3)
        & (col("frequency_score") <= 2)
        & (col("monetary_score") <= 2),
        "Potential Loyalists",
    )
    .when(
        (col("recency_score") >= 4)
        & (col("frequency_score") >= 3)
        & (col("monetary_score") <= 2),
        "New Customers",
    )
    .when(
        (col("recency_score") >= 3)
        & (col("frequency_score") >= 1)
        & (col("monetary_score") >= 3),
        "Need Attention",
    )
    .when(
        (col("recency_score") <= 2)
        & (col("frequency_score") >= 3)
        & (col("monetary_score") >= 3),
        "At Risk",
    )
    .when(
        (col("recency_score") <= 2)
        & (col("frequency_score") <= 2)
        & (col("monetary_score") >= 3),
        "Cant Lose Them",
    )
    .when(
        (col("recency_score") <= 1)
        & (col("frequency_score") >= 2)
        & (col("monetary_score") >= 2),
        "Hibernating",
    )
    .when(
        (col("recency_score") <= 2)
        & (col("frequency_score") <= 2)
        & (col("monetary_score") <= 2),
        "Lost",
    )
    .otherwise("Others"),
)

In [139]:
rfm_scored = rfm_scored.withColumns(
    {
        # Overall RFM score (average of three scores)
        "rfm_overall_score": (
            col("recency_score") + col("frequency_score") + col("monetary_score")
        )
        / lit(3),
        # RFM category (Simple classification)
        "rfm_category": when(
            (col("recency_score") + col("frequency_score") + col("monetary_score"))
            >= 12,
            "High Value",
        )
        .when(
            (col("recency_score") + col("frequency_score") + col("monetary_score"))
            >= 9,
            "Medium Value",
        )
        .otherwise("Low Value"),
        # Engagement level
        "engagement_level": when(col("recency_score") >= 4, "Highly Engaged")
        .when(col("recency_score") >= 3, "Moderately Engaged")
        .otherwise("Low Engagement"),
        # Purchase behavior
        "purchase_behavior": when(col("frequency_score") >= 4, "Frequent Buyer")
        .when(col("frequency_score") >= 2, "Occasional Buyer")
        .otherwise("Rare Buyer"),
        # Spending pattern
        "spending_pattern": when(col("monetary_score") >= 4, "High Spender")
        .when(col("monetary_score") >= 2, "Average Spender")
        .otherwise("Low Spender"),
        # Risk flag
        "churn_risk": when(
            col("customer_segment_label").isin(
                "At Risk", "Cant Lose Them", "Hibernating", "Lost"
            ),
            "High Risk",
        )
        .when(col("customer_segment_label") == "Need Attention", "Medium Risk")
        .otherwise("Low Risk"),
        # Handle nulls
        "days_since_last_order": coalesce(col("days_since_last_order"), lit(999)),
        "total_orders_rfm": coalesce(col("total_orders_rfm"), lit(0)),
        "total_revenue_rfm": coalesce(col("total_revenue_rfm"), lit(0.0)),
    }
)

dataframes["rfm_segmentation"] = rfm_scored

In [140]:
dataframes["customers"] = (
    dataframes["customers"]
    .join(
        rfm_scored.select(
            col("customer_id").alias("rfm_customer_id"),
            col("recency_score").alias("rfm_recency_score"),
            col("frequency_score").alias("rfm_frequency_score"),
            col("monetary_score").alias("rfm_monetary_score"),
            col("rfm_segment").alias("rfm_segment_code"),
            col("customer_segment_label").alias("rfm_customer_segment_label"),
            col("rfm_overall_score").alias("rfm_overall_score_value"),
            col("rfm_category").alias("rfm_category_tier"),
            col("churn_risk").alias("rfm_churn_risk"),
        ),
        dataframes["customers"]["customer_id"] == col("rfm_customer_id"),
        "left",
    )
    .drop("rfm_customer_id")
    .withColumnRenamed("rfm_recency_score", "recency_score")
    .withColumnRenamed("rfm_frequency_score", "frequency_score")
    .withColumnRenamed("rfm_monetary_score", "monetary_score")
    .withColumnRenamed("rfm_segment_code", "rfm_segment")
    .withColumnRenamed("rfm_customer_segment_label", "customer_segment_label")
    .withColumnRenamed("rfm_overall_score_value", "rfm_overall_score")
    .withColumnRenamed("rfm_category_tier", "rfm_category")
    .withColumnRenamed("rfm_churn_risk", "churn_risk")
).dropDuplicates(["customer_id"])

In [141]:
rfm_segment_summary = (
    rfm_scored.groupBy("customer_segment_label")
    .agg(
        count("customer_id").alias("customer_count"),
        spark_avg("total_revenue_rfm").alias("avg_revenue"),
        spark_avg("total_orders_rfm").alias("avg_orders"),
        spark_avg("days_since_last_order").alias("avg_days_since_order"),
        spark_avg("rfm_overall_score").alias("avg_rfm_score"),
    )
    .orderBy(col("customer_count").desc())
)

dataframes["rfm_segment_summary"] = rfm_segment_summary

## Product Affinity & Association

In [142]:
order_products = (
    dataframes["order_items"]
    .filter(col("order_id").isNotNull() & col("product_id").isNotNull())
    .select("order_id", "product_id")
    .distinct()
)

In [143]:
product_pairs = (
    order_products.alias("a")
    .join(
        order_products.alias("b"),
        (col("a.order_id") == col("b.order_id"))
        & (
            col("a.product_id") < col("b.product_id")
        ),  # Avoid duplicates and self-pairs
        "inner",
    )
    .select(
        col("a.product_id").alias("product_a_id"),
        col("b.product_id").alias("product_b_id"),
        col("a.order_id"),
    )
)

In [144]:
product_affinity = product_pairs.groupBy("product_a_id", "product_b_id").agg(
    countDistinct("order_id").alias("co_occurrence_count")
)

In [145]:
total_orders = dataframes["orders"].count()

In [146]:
product_occurrence = order_products.groupBy("product_id").agg(
    countDistinct("order_id").alias("product_order_count")
)

In [147]:
product_affinity = product_affinity.join(
    product_occurrence.select(
        col("product_id").alias("prod_a_id"),
        col("product_order_count").alias("product_a_count"),
    ),
    col("product_a_id") == col("prod_a_id"),
    "left",
).drop("prod_a_id")

In [148]:
product_affinity = product_affinity.join(
    product_occurrence.select(
        col("product_id").alias("prod_b_id"),
        col("product_order_count").alias("product_b_count"),
    ),
    col("product_b_id") == col("prod_b_id"),
    "left",
).drop("prod_b_id")

In [149]:
product_affinity = product_affinity.withColumns(
    {
        # Handle nulls first
        "co_occurrence_count": coalesce(col("co_occurrence_count"), lit(0)),
        "product_a_count": coalesce(col("product_a_count"), lit(0)),
        "product_b_count": coalesce(col("product_b_count"), lit(0)),
    }
)

In [150]:
product_affinity = product_affinity.withColumns(
    {
        # Support: P(A and B) = co-occurrence / total_orders
        "support": when(
            (col("co_occurrence_count") > 0) & (lit(total_orders) > 0),
            (col("co_occurrence_count") / lit(total_orders)) * 100,
        ).otherwise(lit(0.0)),
        # Confidence: P(B|A) = co-occurrence / product_a_count
        "confidence_a_to_b": when(
            (col("co_occurrence_count") > 0) & (col("product_a_count") > 0),
            (col("co_occurrence_count") / col("product_a_count")) * 100,
        ).otherwise(lit(0.0)),
        # Confidence: P(A|B) = co-occurrence / product_b_count
        "confidence_b_to_a": when(
            (col("co_occurrence_count") > 0) & (col("product_b_count") > 0),
            (col("co_occurrence_count") / col("product_b_count")) * 100,
        ).otherwise(lit(0.0)),
        # P(B) = product_b_count / total_orders
        "prob_b": when(
            (col("product_b_count") > 0) & (lit(total_orders) > 0),
            col("product_b_count") / lit(total_orders),
        ).otherwise(lit(0.0)),
        # P(A) = product_a_count / total_orders
        "prob_a": when(
            (col("product_a_count") > 0) & (lit(total_orders) > 0),
            col("product_a_count") / lit(total_orders),
        ).otherwise(lit(0.0)),
    }
)

In [151]:
product_affinity = product_affinity.withColumns(
    {
        # Lift: confidence / P(B)
        "lift_a_to_b": when(
            (col("confidence_a_to_b") > 0) & (col("prob_b") > 0),
            (col("confidence_a_to_b") / 100) / col("prob_b"),
        ).otherwise(lit(0.0)),
        # Lift: confidence / P(A)
        "lift_b_to_a": when(
            (col("confidence_b_to_a") > 0) & (col("prob_a") > 0),
            (col("confidence_b_to_a") / 100) / col("prob_a"),
        ).otherwise(lit(0.0)),
    }
)

In [152]:
product_affinity = product_affinity.withColumn(
    "avg_lift", (col("lift_a_to_b") + col("lift_b_to_a")) / lit(2)
)

In [153]:
product_affinity = (
    product_affinity.join(
        dataframes["products"].select(
            col("product_id").alias("prod_a_join_id"),
            col("product_name").alias("product_a_name"),
            col("category").alias("product_a_category"),
        ),
        col("product_a_id") == col("prod_a_join_id"),
        "left",
    )
    .drop("prod_a_join_id")
    .join(
        dataframes["products"].select(
            col("product_id").alias("prod_b_join_id"),
            col("product_name").alias("product_b_name"),
            col("category").alias("product_b_category"),
        ),
        col("product_b_id") == col("prod_b_join_id"),
        "left",
    )
    .drop("prod_b_join_id")
)

In [154]:
product_affinity = product_affinity.withColumns(
    {
        # Affinity strength based on lift
        "affinity_strength": when(col("avg_lift") >= 3.0, "Very Strong")
        .when(col("avg_lift") >= 2.0, "Strong")
        .when(col("avg_lift") >= 1.5, "Moderate")
        .when(col("avg_lift") >= 1.0, "Weak")
        .otherwise("No Affinity"),
        # Cross-category indicator
        "is_cross_category": when(
            col("product_a_category") != col("product_b_category"), lit(True)
        ).otherwise(lit(False)),
        # Affinity score (composite metric)
        "affinity_score": (
            # Support weight (30%)
            (col("support") / 10) * lit(0.3)
            +
            # Lift weight (50%)
            col("avg_lift") * lit(0.5)
            +
            # Frequency weight (20%)
            (col("co_occurrence_count") / 100) * lit(0.2)
        )
        * lit(10),
    }
)

In [155]:
product_affinity_filtered = product_affinity.filter(
    (col("co_occurrence_count") >= 3)  # At least 3 co-occurrences
    & (col("avg_lift") >= 1.0)  # Lift greater than random
)

In [156]:
dataframes["product_affinity"] = product_affinity_filtered

In [157]:
top_product_pairs = product_affinity_filtered.orderBy(
    col("affinity_score").desc()
).limit(100)

dataframes["top_product_pairs"] = top_product_pairs

In [158]:
window_product_a = Window.partitionBy("product_a_id").orderBy(
    col("affinity_score").desc()
)

product_recommendations = (
    product_affinity_filtered.withColumn("rank", row_number().over(window_product_a))
    .filter(col("rank") <= 5)
    .groupBy("product_a_id", "product_a_name")
    .agg(
        expr("collect_list(product_b_name)").alias("recommended_products"),
        spark_avg("affinity_score").alias("avg_affinity_score"),
        count("product_b_id").alias("recommendation_count"),
    )
)

dataframes["product_recommendations"] = product_recommendations

In [159]:
category_affinity = (
    product_affinity_filtered.groupBy("product_a_category", "product_b_category")
    .agg(
        count("*").alias("pair_count"),
        spark_avg("avg_lift").alias("avg_lift_between_categories"),
        spark_avg("support").alias("avg_support"),
        spark_sum("co_occurrence_count").alias("total_co_occurrences"),
    )
    .filter(col("product_a_category") != col("product_b_category"))
    .orderBy(col("avg_lift_between_categories").desc())
)

dataframes["category_affinity"] = category_affinity

## Global Aggregations

In [160]:
# ============================================================================
# M. Global Aggregations (entire dataset)
# ============================================================================

# Calculate global metrics across entire dataset
global_metrics = {}

# Total revenue all time
global_metrics["total_revenue_all_time"] = (
    dataframes["orders"]
    .filter(col("total_amount").isNotNull() & (col("total_amount") > 0))
    .agg(spark_sum("total_amount").alias("value"))
    .collect()[0]["value"]
    or 0.0
)

# Total orders all time
global_metrics["total_orders_all_time"] = dataframes["orders"].count()

# Total customers all time
global_metrics["total_customers_all_time"] = dataframes["customers"].count()

# Average order value global
global_metrics["avg_order_value_global"] = (
    dataframes["orders"]
    .filter(col("total_amount").isNotNull() & (col("total_amount") > 0))
    .agg(spark_avg("total_amount").alias("value"))
    .collect()[0]["value"]
    or 0.0
)

# Total products in catalog
global_metrics["total_products_catalog"] = dataframes["products"].count()

# Average customer lifetime value global
global_metrics["avg_customer_lifetime_value_global"] = (
    dataframes["customers"]
    .filter(col("customer_lifetime_value").isNotNull())
    .agg(spark_avg("customer_lifetime_value").alias("value"))
    .collect()[0]["value"]
    or 0.0
)

# Overall conversion rate (sessions with orders / total sessions)
total_sessions = dataframes["customer_sessions"].count()
converted_sessions = (
    dataframes["customer_sessions"].filter(col("conversion_flag") == 1).count()
)

global_metrics["overall_conversion_rate"] = (
    (converted_sessions / total_sessions * 100) if total_sessions > 0 else 0.0
)

# Overall cart abandonment rate
total_carts = dataframes["shopping_cart"].select("cart_id").distinct().count()
abandoned_carts = (
    dataframes["shopping_cart"]
    .filter(
        col("cart_status").isNotNull()
        & ~lower(col("cart_status")).isin("purchased", "completed", "ordered")
    )
    .select("cart_id")
    .distinct()
    .count()
)

global_metrics["overall_cart_abandonment_rate"] = (
    (abandoned_carts / total_carts * 100) if total_carts > 0 else 0.0
)

# Average delivery time in days
global_metrics["avg_delivery_time_days"] = (
    dataframes["orders"]
    .filter(col("order_shipped_at").isNotNull() & col("order_delivered_at").isNotNull())
    .withColumn(
        "delivery_time", datediff(col("order_delivered_at"), col("order_shipped_at"))
    )
    .filter(col("delivery_time") >= 0)
    .agg(spark_avg("delivery_time").alias("value"))
    .collect()[0]["value"]
    or 0.0
)

# Overall customer satisfaction (average rating)
global_metrics["overall_customer_satisfaction"] = (
    dataframes["reviews"]
    .filter(col("rating").isNotNull() & (col("rating") > 0))
    .agg(spark_avg("rating").alias("value"))
    .collect()[0]["value"]
    or 0.0
)

# Additional useful global metrics
# Total units sold all time
global_metrics["total_units_sold_all_time"] = (
    dataframes["order_items"]
    .filter(col("quantity").isNotNull() & (col("quantity") > 0))
    .agg(spark_sum("quantity").alias("value"))
    .collect()[0]["value"]
    or 0
)

# Total active customers (with at least one order)
global_metrics["total_active_customers"] = (
    dataframes["orders"].select("customer_id").distinct().count()
)

# Customer activation rate
global_metrics["customer_activation_rate"] = (
    (
        global_metrics["total_active_customers"]
        / global_metrics["total_customers_all_time"]
        * 100
    )
    if global_metrics["total_customers_all_time"] > 0
    else 0.0
)

# Average items per order
global_metrics["avg_items_per_order"] = (
    dataframes["order_items"]
    .filter(col("quantity").isNotNull() & (col("quantity") > 0))
    .groupBy("order_id")
    .agg(spark_sum("quantity").alias("order_items"))
    .agg(spark_avg("order_items").alias("value"))
    .collect()[0]["value"]
    or 0.0
)

# Total reviews submitted
global_metrics["total_reviews_all_time"] = dataframes["reviews"].count()

# Review participation rate
global_metrics["review_participation_rate"] = (
    (
        global_metrics["total_reviews_all_time"]
        / global_metrics["total_orders_all_time"]
        * 100
    )
    if global_metrics["total_orders_all_time"] > 0
    else 0.0
)

# Total suppliers
global_metrics["total_suppliers"] = dataframes["suppliers"].count()

# Average products per supplier
global_metrics["avg_products_per_supplier"] = (
    dataframes["products"]
    .filter(col("supplier_id").isNotNull())
    .groupBy("supplier_id")
    .agg(count("product_id").alias("product_count"))
    .agg(spark_avg("product_count").alias("value"))
    .collect()[0]["value"]
    or 0.0
)

# Total categories
global_metrics["total_categories"] = (
    dataframes["products"]
    .select("category")
    .filter(col("category").isNotNull())
    .distinct()
    .count()
)

# Average price point
global_metrics["avg_product_price"] = (
    dataframes["products"]
    .filter(col("sell_price").isNotNull() & (col("sell_price") > 0))
    .agg(spark_avg("sell_price").alias("value"))
    .collect()[0]["value"]
    or 0.0
)

# Total marketing campaigns
global_metrics["total_marketing_campaigns"] = dataframes["marketing_campaigns"].count()

# Average campaign ROI
global_metrics["avg_campaign_roi"] = (
    dataframes["marketing_campaigns"]
    .filter(col("roi").isNotNull())
    .agg(spark_avg("roi").alias("value"))
    .collect()[0]["value"]
    or 0.0
)

# Total inventory value
global_metrics["total_inventory_value"] = (
    dataframes["inventory"]
    .join(
        dataframes["products"].select(
            col("product_id").alias("inv_prod_id"), "cost_price"
        ),
        dataframes["inventory"]["product_id"] == col("inv_prod_id"),
        "left",
    )
    .filter(
        col("stock_quantity").isNotNull()
        & col("cost_price").isNotNull()
        & (col("stock_quantity") > 0)
        & (col("cost_price") > 0)
    )
    .withColumn("inventory_value", col("stock_quantity") * col("cost_price"))
    .agg(spark_sum("inventory_value").alias("value"))
    .collect()[0]["value"]
    or 0.0
)

# Average order processing time (days)
global_metrics["avg_order_processing_days"] = (
    dataframes["orders"]
    .filter(col("order_placed_at").isNotNull() & col("order_shipped_at").isNotNull())
    .withColumn(
        "processing_time",
        datediff(col("order_shipped_at"), to_date(col("order_placed_at"))),
    )
    .filter(col("processing_time") >= 0)
    .agg(spark_avg("processing_time").alias("value"))
    .collect()[0]["value"]
    or 0.0
)

# Create DataFrame from global metrics
global_aggregations_data = [(k, float(v)) for k, v in global_metrics.items()]
global_aggregations_df = spark.createDataFrame(
    global_aggregations_data, ["metric_name", "metric_value"]
).withColumn("calculated_at", lit("2025-11-14 16:34:39"))

dataframes["global_aggregations"] = global_aggregations_df

# Create a summary view with formatted metrics
print("\n" + "=" * 80)
print("GLOBAL BUSINESS METRICS SUMMARY")
print("=" * 80)
print(f"\nRevenue & Orders:")
print(
    f"  Total Revenue (All Time):           ${global_metrics['total_revenue_all_time']:,.2f}"
)
print(
    f"  Total Orders (All Time):            {global_metrics['total_orders_all_time']:,}"
)
print(
    f"  Average Order Value:                ${global_metrics['avg_order_value_global']:,.2f}"
)
print(
    f"  Total Units Sold:                   {global_metrics['total_units_sold_all_time']:,}"
)

print(f"\nCustomers:")
print(
    f"  Total Customers:                    {global_metrics['total_customers_all_time']:,}"
)
print(
    f"  Active Customers:                   {global_metrics['total_active_customers']:,}"
)
print(
    f"  Customer Activation Rate:           {global_metrics['customer_activation_rate']:.2f}%"
)
print(
    f"  Avg Customer Lifetime Value:        ${global_metrics['avg_customer_lifetime_value_global']:,.2f}"
)

print(f"\nConversion & Engagement:")
print(
    f"  Overall Conversion Rate:            {global_metrics['overall_conversion_rate']:.2f}%"
)
print(
    f"  Cart Abandonment Rate:              {global_metrics['overall_cart_abandonment_rate']:.2f}%"
)
print(
    f"  Customer Satisfaction (Avg Rating): {global_metrics['overall_customer_satisfaction']:.2f}/5.0"
)
print(
    f"  Review Participation Rate:          {global_metrics['review_participation_rate']:.2f}%"
)

print(f"\nOperations:")
print(
    f"  Avg Order Processing Time:          {global_metrics['avg_order_processing_days']:.1f} days"
)
print(
    f"  Avg Delivery Time:                  {global_metrics['avg_delivery_time_days']:.1f} days"
)
print(
    f"  Avg Items per Order:                {global_metrics['avg_items_per_order']:.1f}"
)

print(f"\nCatalog & Inventory:")
print(
    f"  Total Products in Catalog:          {global_metrics['total_products_catalog']:,}"
)
print(f"  Total Categories:                   {global_metrics['total_categories']:,}")
print(
    f"  Average Product Price:              ${global_metrics['avg_product_price']:,.2f}"
)
print(
    f"  Total Inventory Value:              ${global_metrics['total_inventory_value']:,.2f}"
)

print(f"\nSuppliers & Marketing:")
print(f"  Total Suppliers:                    {global_metrics['total_suppliers']:,}")
print(
    f"  Avg Products per Supplier:          {global_metrics['avg_products_per_supplier']:.1f}"
)
print(
    f"  Total Marketing Campaigns:          {global_metrics['total_marketing_campaigns']:,}"
)
print(
    f"  Average Campaign ROI:               {global_metrics['avg_campaign_roi']:.2f}%"
)

print("\n" + "=" * 80)
print("Global aggregations completed!")
print(f"Metrics stored in: dataframes['global_aggregations']")
print("=" * 80 + "\n")

# Show the dataframe
dataframes["global_aggregations"].show(50, False)


GLOBAL BUSINESS METRICS SUMMARY

Revenue & Orders:
  Total Revenue (All Time):           $1,250,873.80
  Total Orders (All Time):            1,185
  Average Order Value:                $1,055.59
  Total Units Sold:                   179,520

Customers:
  Total Customers:                    980
  Active Customers:                   686
  Customer Activation Rate:           70.00%
  Avg Customer Lifetime Value:        $4,155.11

Conversion & Engagement:
  Overall Conversion Rate:            9.26%
  Cart Abandonment Rate:              7.95%
  Customer Satisfaction (Avg Rating): 3.74/5.0
  Review Participation Rate:          228.10%

Operations:
  Avg Order Processing Time:          908.8 days
  Avg Delivery Time:                  450.8 days
  Avg Items per Order:                135.3

Catalog & Inventory:
  Total Products in Catalog:          905
  Total Categories:                   115
  Average Product Price:              $468.25
  Total Inventory Value:              $25,486,957.60

S

+----------------------------------+--------------------+-------------------+
|metric_name                       |metric_value        |calculated_at      |
+----------------------------------+--------------------+-------------------+
|total_revenue_all_time            |1250873.7999999984  |2025-11-14 16:34:39|
|total_orders_all_time             |1185.0              |2025-11-14 16:34:39|
|total_customers_all_time          |980.0               |2025-11-14 16:34:39|
|avg_order_value_global            |1055.5897046413488  |2025-11-14 16:34:39|
|total_products_catalog            |905.0               |2025-11-14 16:34:39|
|avg_customer_lifetime_value_global|4155.113102040814   |2025-11-14 16:34:39|
|overall_conversion_rate           |9.256801592568015   |2025-11-14 16:34:39|
|overall_cart_abandonment_rate     |7.952622673434856   |2025-11-14 16:34:39|
|avg_delivery_time_days            |450.8478260869565   |2025-11-14 16:34:39|
|overall_customer_satisfaction     |3.73621901590825    |2025-11

In [161]:
dataframes["order_items"] = dataframes["order_items"].dropDuplicates(["order_item_id"])
dataframes["payments"] = dataframes["payments"].dropDuplicates(["payment_id"])

In [162]:
dataframes

{'customer_sessions': DataFrame[session_id: string, customer_id: int, session_start: timestamp, session_end: timestamp, device_type: string, referrer_source: string, pages_viewed: int, products_viewed: int, conversion_flag: int, cart_abandonment_flag: int, session_duration_seconds: bigint, session_duration_minutes: bigint, session_duration_hours: bigint, pages_per_minute: double, products_per_page: double, items_added_to_cart: bigint, cart_value: double, orders_from_session: bigint, total_pages_viewed: int, total_products_viewed: int, converted: int, abandoned: int, cart_add_rate: double, avg_cart_item_value: double, session_engagement_score: double, session_type: string],
 'customers': DataFrame[customer_id: int, gender: string, date_of_birth: date, account_status: string, city: string, state_province: string, postal_code: string, country: string, account_created_at: timestamp, last_login_date: date, is_active: boolean, order_recency_days: int, order_frequency: bigint, order_total_spe

In [163]:
primary_keys = {
    "customer_sessions": ["session_id"],
    "customers": ["customer_id"],
    "inventory": [
        "inventory_id"
    ],  # could also be product_id + supplier_id depending on business rules
    "marketing_campaigns": ["campaign_id"],
    "order_items": ["order_item_id"],
    "orders": ["order_id"],
    "payments": ["payment_id"],
    "products": ["product_id"],
    "reviews": ["review_id"],
    "shopping_cart": ["cart_id"],
    "suppliers": ["supplier_id"],
    "wishlist": ["wishlist_id"],
    "categories": ["category"],
    "daily_aggregations": ["order_date"],
    "weekly_aggregations": ["year_week"],
    "monthly_aggregations": ["year_month"],
    "country_aggregations": ["country"],
    "state_aggregations": ["country", "state_province"],
    "city_aggregations": ["country", "state_province", "city"],
    "cart_abandonment_analysis": ["cart_id"],
    "product_inventory_health": ["product_id"],
    "supplier_inventory_health": ["supplier_id"],
    "rfm_segmentation": ["customer_id"],
    "rfm_segment_summary": ["customer_segment_label"],
    "product_affinity": ["product_a_id", "product_b_id"],
    "top_product_pairs": ["product_a_id", "product_b_id"],
    "product_recommendations": ["product_a_id"],
    "category_affinity": ["product_a_category", "product_b_category"],
    "global_aggregations": ["metric_name"],
}

In [164]:
from pyspark.sql import functions as F

for name, pk in primary_keys.items():
    df = dataframes[name]
    count_dupes = df.groupBy(pk).count().filter("count > 1").count()
    print(f"{name}: {count_dupes} duplicate PK groups remaining")

customer_sessions: 0 duplicate PK groups remaining
customers: 0 duplicate PK groups remaining
inventory: 0 duplicate PK groups remaining
marketing_campaigns: 0 duplicate PK groups remaining
order_items: 0 duplicate PK groups remaining
orders: 0 duplicate PK groups remaining
payments: 0 duplicate PK groups remaining
products: 0 duplicate PK groups remaining
reviews: 0 duplicate PK groups remaining
shopping_cart: 0 duplicate PK groups remaining
suppliers: 0 duplicate PK groups remaining
wishlist: 0 duplicate PK groups remaining
categories: 0 duplicate PK groups remaining
daily_aggregations: 0 duplicate PK groups remaining
weekly_aggregations: 0 duplicate PK groups remaining
monthly_aggregations: 0 duplicate PK groups remaining
country_aggregations: 0 duplicate PK groups remaining
state_aggregations: 0 duplicate PK groups remaining
city_aggregations: 0 duplicate PK groups remaining
cart_abandonment_analysis: 0 duplicate PK groups remaining
product_inventory_health: 0 duplicate PK groups r

global_aggregations: 0 duplicate PK groups remaining


In [165]:
# for name, df in cleaned_dataframes.items():
#     if name in primary_keys:
#         df_duplicate_keys = df.groupBy(primary_keys[name]).count().filter(F.col("count") > 1)
#         df_duplicates = df.join(
#             df_duplicate_keys.drop("count"), on=primary_keys[name], how="left_semi"
#         )
#         df_duplicates.show(truncate=False)

In [166]:
conn = psycopg2.connect(
    host=os.getenv("POSTGRES_SERVER", "localhost"),
    port=os.getenv("POSTGRES_PORT", "5432"),
    database=os.getenv("POSTGRES_DATABASE_NAME"),
    user=os.getenv("POSTGRES_USER"),
    password=os.getenv("POSTGRES_PASSWORD"),
)
conn.autocommit = True
cursor = conn.cursor()

# Define table mappings
table_mappings = {
    "customers": "agg_customers",
    "orders": "agg_orders",
    "products": "agg_products",
    "marketing_campaigns": "agg_marketing_campaigns",
    "suppliers": "agg_suppliers",
    "inventory": "agg_inventory",
    "customer_sessions": "agg_customer_sessions",
    "wishlist": "agg_wishlist",
    "shopping_cart": "agg_shopping_cart",
    "reviews": "agg_reviews",
    "order_items": "agg_order_items",
    "payments": "agg_payments",
    "daily_aggregations": "agg_daily_aggregations",
    "weekly_aggregations": "agg_weekly_aggregations",
    "monthly_aggregations": "agg_monthly_aggregations",
    "country_aggregations": "agg_country_aggregations",
    "state_aggregations": "agg_state_aggregations",
    "city_aggregations": "agg_city_aggregations",
    "categories": "agg_categories",
    "cart_abandonment_analysis": "agg_cart_abandonment_analysis",
    "product_inventory_health": "agg_product_inventory_health",
    "supplier_inventory_health": "agg_supplier_inventory_health",
    "rfm_segmentation": "agg_rfm_segmentation",
    "rfm_segment_summary": "agg_rfm_segment_summary",
    "product_affinity": "agg_product_affinity",
    "top_product_pairs": "agg_top_product_pairs",
    "product_recommendations": "agg_product_recommendations",
    "category_affinity": "agg_category_affinity",
    "global_aggregations": "agg_global_aggregations",
}

print("Clearing existing data...")
for table_name in table_mappings.values():
    cursor.execute(f"TRUNCATE TABLE {table_name} CASCADE")

print("\nLoading data to PostgreSQL...")
jdbc_url = f"jdbc:postgresql://{os.getenv('POSTGRES_SERVER', 'localhost')}:{os.getenv('POSTGRES_PORT', '5432')}/pulse"

try:
    for df_name, table_name in table_mappings.items():
        if df_name in dataframes:
            print(f"  Loading {table_name}...")

            dataframes[df_name].write.format("jdbc").option("url", jdbc_url).option(
                "dbtable", table_name
            ).option("user", os.getenv("POSTGRES_USER")).option(
                "password", os.getenv("POSTGRES_PASSWORD")
            ).option(
                "driver", "org.postgresql.Driver"
            ).mode(
                "append"
            ).save()

            print(f"    ✓ Done")

    print("\n✅ All data loaded successfully!")

except Exception as e:
    print(f"\n❌ Error: {str(e)}")
    raise
finally:
    cursor.close()
    conn.close()

spark.stop()

Clearing existing data...

Loading data to PostgreSQL...
  Loading agg_customers...
    ✓ Done
  Loading agg_orders...
    ✓ Done
  Loading agg_products...


    ✓ Done
  Loading agg_marketing_campaigns...
    ✓ Done
  Loading agg_suppliers...


    ✓ Done
  Loading agg_inventory...
    ✓ Done
  Loading agg_customer_sessions...
    ✓ Done
  Loading agg_wishlist...
    ✓ Done
  Loading agg_shopping_cart...
    ✓ Done
  Loading agg_reviews...
    ✓ Done
  Loading agg_order_items...
    ✓ Done
  Loading agg_payments...
    ✓ Done
  Loading agg_daily_aggregations...


25/11/27 23:45:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/27 23:45:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/27 23:45:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/27 23:45:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/27 23:45:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/27 23:45:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/27 2

    ✓ Done
  Loading agg_weekly_aggregations...


...

    ✓ Done
  Loading agg_monthly_aggregations...


25/11/27 23:46:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/27 23:46:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/27 23:46:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/27 23:46:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/27 23:46:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/27 23:46:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/27 2

    ✓ Done
  Loading agg_country_aggregations...
    ✓ Done
  Loading agg_state_aggregations...
    ✓ Done
  Loading agg_city_aggregations...
    ✓ Done
  Loading agg_categories...


25/11/27 23:46:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/27 23:46:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/27 23:46:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/27 23:46:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/27 23:46:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/27 23:46:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/27 2

    ✓ Done
  Loading agg_cart_abandonment_analysis...
    ✓ Done
  Loading agg_product_inventory_health...
    ✓ Done
  Loading agg_supplier_inventory_health...
    ✓ Done
  Loading agg_rfm_segmentation...
    ✓ Done
  Loading agg_rfm_segment_summary...
    ✓ Done
  Loading agg_product_affinity...
    ✓ Done
  Loading agg_top_product_pairs...
    ✓ Done
  Loading agg_product_recommendations...
    ✓ Done
  Loading agg_category_affinity...
    ✓ Done
  Loading agg_global_aggregations...
    ✓ Done

✅ All data loaded successfully!


In [167]:
spark.stop()